# DQN
传统的强化学习算法会使用表格的形式存储状态值函数 $V(s)$ 或状态动作值函数 $Q(s,a)$，但是这样的方法存在很大的局限性。例如：现实中的强化学习任务所面临的状态空间往往是连续的，存在无穷多个状态，在这种情况下，就不能再使用表格对值函数进行存储。值函数近似利用函数直接拟合状态值函数或状态动作值函数，减少了对存储空间的要求，有效地解决了这个问题。

为了在连续的状态空间中计算价值函数 $Q^{\pi}(s,a)$，我们可以用一个函数 $Q_{\phi}(\boldsymbol{s},\boldsymbol{a})$ 来表示近似计算，称为`价值函数近似(Value Function Approximation)`。
$$
Q_{\phi}(\boldsymbol{s}, \boldsymbol{a}) \approx Q^{\pi}(s, a)
$$

其中
* $\boldsymbol{s},\boldsymbol{a}$ 分别是状态 $s$ 和动作 $a$ 的向量表示，
* 函数 $Q_{\phi}(\boldsymbol{s}, \boldsymbol{a})$ 通常是一个参数为 $\phi$ 的函数，比如`神经网络`，输出为一个实数，称为`Q 网络(Q-network)`。

深度Q网络（Deep Q-Network，DQN）算法的核心是维护 Q 函数并使用其进行决策。$Q^{\pi}(s,a)$ 为在该策略 $\pi$ 下的动作价值函数，每次到达一个状态 $s_t$ 之后，遍历整个动作空间，使用让 $Q^{\pi}(s,a)$ 最大的动作作为策略：
$$
a_{t}=\underset{a}{\arg \max } ~Q^{\pi}\left(s_{t}, a\right)
$$
DQN采用贝尔曼方程来迭代更新 $Q^{\pi}(s,a)$ ：
$$
Q^{\pi}\left(s_{t}, a_{t}\right) \leftarrow Q^{\pi}\left(s_{t}, a_{t}\right)+\alpha\left(r_{t}+\gamma \max _{a} Q^{\pi}\left(s_{t+1}, a\right)-Q^{\pi}\left(s_{t}, a_{t}\right)\right)
$$
通常在简单任务上，使用全连接神经网络（fully connected neural network）来拟合 $Q^{\pi}$，但是在较为复杂的任务上（如玩雅达利游戏），会使用卷积神经网络来拟合从图像到价值函数的映射。由于DQN的这种表达形式只能处理有限个动作值，因此其通常用于处理离散动作空间的任务。

## State Value Function

**Q-learning 是 `value-based` 的方法。在 value-based 的方法里面，我们学习的不是策略，我们要学习的是一个 `critic(评论家)`。**评论家要做的事情是评价现在的行为有多好或是有多不好。假设有一个演员(actor) $\pi$ ，评论家就是来评价这个演员的策略 $\pi$  好还是不好，即 `Policy Evaluation(策略评估)`。

> 注：「李宏毅深度强化学习」课程提到的 Q-learning，其实是 DQN。
>
> DQN 是指基于深度学习的 Q-learning 算法，主要结合了`价值函数近似(Value Function Approximation)`与神经网络技术，并采用了目标网络和经历回放的方法进行网络的训练。
>
> 在 Q-learning 中，我们使用表格来存储每个状态 s 下采取动作 a 获得的奖励，即状态-动作值函数 $Q(s,a)$。然而，这种方法在状态量巨大甚至是连续的任务中，会遇到维度灾难问题，往往是不可行的。因此，DQN 采用了价值函数近似的表示方法。

举例来说，有一种评论家叫做 `state value function(状态价值函数)`。状态价值函数的意思就是说，假设演员叫做 $\pi$，拿 $\pi$  跟环境去做互动。假设 $\pi$  看到了某一个状态 s，如果在玩 Atari 游戏的话，状态 s 是某一个画面，看到某一个画面的时候，接下来一直玩到游戏结束，期望的累积奖励有多大。所以 $V^{\pi}$ 是一个函数，这个函数输入一个状态，然后它会输出一个标量( scalar)。这个标量代表说，$\pi$ 这个演员看到状态 s 的时候，接下来预期到游戏结束的时候，它可以得到多大的值。

![](img/6.1.png ':size=550')

举个例子，假设你是玩 space invader 的话，

* 左边这个状态 s，这个游戏画面，$V^{\pi}(s)$  也许会很大，因为还有很多的怪可以杀， 所以你会得到很大的分数。一直到游戏结束的时候，你仍然有很多的分数可以吃。
* 右边这种情况得到的 $V^{\pi}(s)$ 可能就很小，因为剩下的怪也不多了，并且红色的防护罩已经消失了，所以可能很快就会死掉。所以接下来得到预期的奖励，就不会太大。

这边需要强调的一个点是说，评论家都是绑一个演员的，评论家没有办法去凭空去评价一个状态的好坏，它所评价的东西是在给定某一个状态的时候， 假设接下来互动的演员是 $\pi$，那我会得到多少奖励。因为就算是给同样的状态，你接下来的 $\pi$ 不一样，你得到的奖励也是不一样的。

举例来说，在左边的情况，假设是一个正常的 $\pi$，它可以杀很多怪，那假设它是一个很弱的 $\pi$，它就站在原地不动，然后马上就被射死了，那你得到的 $V^\pi(s)$ 还是很小。所以评论家的输出值取决于状态和演员。所以评论家其实都要绑一个演员，它是在衡量某一个演员的好坏，而不是衡量一个状态的好坏。这边要强调一下，评论家的输出是跟演员有关的，状态的价值其实取决于你的演员，当演员变的时候，状态价值函数的输出其实也是会跟着改变的。

### State Value Function Estimation

**怎么衡量这个状态价值函数 $V^{\pi}(s)$ 呢？**有两种不同的做法：MC-based 的方法和 TD-based 的方法。

` Monte-Carlo(MC)-based`的方法就是让演员去跟环境做互动，要看演员好不好，我们就让演员去跟环境做互动，给评论家看。然后，评论家就统计说，

* 演员如果看到状态 $s_a$，接下来的累积奖励会有多大。
* 如果它看到状态 $s_b$，接下来的累积奖励会有多大。

但是实际上，我们不可能把所有的状态通通都扫过。如果是玩 Atari 游戏的话，状态是图像，你没有办法把所有的状态通通扫过。所以实际上 $V^{\pi}(s)$ 是一个网络。对一个网络来说，就算输入状态是从来都没有看过的，它也可以想办法估测一个值。

![](img/6.2.png ':size=350')

怎么训练这个网络呢？因为如果在状态 $s_a$，接下来的累积奖励就是 $G_a$。也就是说，对这个价值函数来说，如果输入是状态 $s_a$，正确的输出应该是 $G_a$。如果输入状态 $s_b$，正确的输出应该是值 $G_b$。**所以在训练的时候， 它就是一个 `回归问题(regression problem)`。**网络的输出就是一个值，你希望在输入 $s_a$ 的时候，输出的值跟 $G_a$ 越近越好，输入 $s_b$ 的时候，输出的值跟 $G_b$ 越近越好。接下来把网络训练下去，就结束了。这是 MC-based 的方法。

![](img/6.3.png ':size=550')

**第二个方法是`Temporal-difference(时序差分)` 的方法， `即 TD-based ` 的方法。**

在 MC-based 的方法中，每次我们都要算累积奖励，也就是从某一个状态 $s_a$ 一直玩到游戏结束的时候，得到的所有奖励的总和。所以你要使用 MC-based 的方法，你必须至少把这个游戏玩到结束。但有些游戏非常长，你要玩到游戏结束才能够更新网络，花的时间太长了，因此我们会采用 TD-based 的方法。

TD-based 的方法不需要把游戏玩到底，只要在游戏的某一个情况，某一个状态 $s_t$ 的时候，采取动作 $a_t$ 得到奖励$r_t$ ，跳到状态 $s_{t+1}$，就可以使用 TD 的方法。

怎么使用 TD 的方法呢？这边是基于以下这个式子：
$$
V^{\pi}\left(s_{t}\right)=V^{\pi}\left(s_{t+1}\right)+r_{t}
$$

假设我们现在用的是某一个策略 $\pi$，在状态 $s_t$，它会采取动作 $a_t$，给我们奖励 $r_t$ ，接下来进入 $s_{t+1}$ 。状态 $s_{t+1}$ 的值跟状态 $s_t$ 的值，它们的中间差了一项 $r_t$。因为你把 $s_{t+1}$ 得到的值加上得到的奖励 $r_t$ 就会等于 $s_t$ 得到的值。有了这个式子以后，你在训练的时候，你并不是直接去估测 V，而是希望你得到的结果 V 可以满足这个式子。

也就是说我们会是这样训练的，我们把 $s_t$ 丢到网络里面，因为 $s_t$ 丢到网络里面会得到 $V^{\pi}(s_t)$，把 $s_{t+1}$ 丢到你的值网络里面会得到 $V^{\pi}(s_{t+1})$，这个式子告诉我们，$V^{\pi}(s_t)$ 减 $V^{\pi}(s_{t+1})$ 的值应该是 $r_t$。然后希望它们两个相减的 loss 跟 $r_t$ 越接近，训练下去，更新 V 的参数，你就可以把 V 函数学习出来。

![](img/6.4.png ':size=500')

**MC 跟 TD 有什么样的差别呢？**

**MC 最大的问题就是方差很大。**因为我们在玩游戏的时候，它本身是有随机性的。所以你可以把 $G_a$ 看成一个随机变量。因为你每次同样走到 $s_a$ 的时候，最后你得到的 $G_a$ 其实是不一样的。你看到同样的状态 $s_a$，最后玩到游戏结束的时候，因为游戏本身是有随机性的，玩游戏的模型搞不好也有随机性，所以你每次得到的 $G_a$ 是不一样的，每一次得到 $G_a$ 的差别其实会很大。为什么它会很大呢？因为 $G_a$ 其实是很多个不同的步骤的奖励的和。假设你每一个步骤都会得到一个奖励，$G_a$ 是从状态 $s_a$ 开始，一直玩到游戏结束，每一个步骤的奖励的和。
$$
\operatorname{Var}[k X]=k^{2} \operatorname{Var}[X]
$$
> Var 是指 variance。 

为了方便说明问题，我们简化一下，假设从状态 $s_a$ 开始，一直玩到游戏结束，各个步骤的奖励相同，均为 $r_a$， $G_a=kr_a$。根据上式，则可得
$$
\operatorname{Var}[G_a]=\operatorname{Var}[kr_a]=k^{2} \operatorname{Var}[r_a]
$$

因此 $G_a$ 的方差相较于某一个状态的奖励是比较大的。

如果用 TD 的话，你是要去最小化这样的一个式子：

![](img/6.5.png ':size=550')

在这中间会有随机性的是 r。因为计算你在 $s_t$ 采取同一个动作，你得到的奖励也不一定是一样的，所以 r 是一个随机变量。但这个随机变量的方差会比 $G_a$ 还要小，因为 $G_a$ 是很多 r 合起来，这边只是某一个 r  而已。$G_a$ 的方差会比较大，r  的方差会比较小。但是这边你会遇到的**一个问题是你这个 V 不一定估得准**。假设你的这个 V 估得是不准的，那你使用这个式子学习出来的结果，其实也会是不准的。所以 MC 跟 TD 各有优劣。**今天其实 TD 的方法是比较常见的，MC 的方法其实是比较少用的。**

![](img/6.6.png ':size=550')

**上图是讲 TD 跟 MC 的差异。**假设有某一个评论家，它去观察某一个策略 $\pi$  跟环境互动的 8 个 episode 的结果。有一个演员 $\pi$  跟环境互动了8 次，得到了8 次玩游戏的结果。接下来这个评论家去估测状态的值。

**我们先计算 $s_b$ 的值。** 状态 $s_b$ 在 8 场游戏里面都有经历过，其中有 6 场得到奖励 1，有 2 场得到奖励 0。所以如果你是要算期望值的话，就算看到状态 $s_b$ 以后得到的奖励，一直到游戏结束的时候得到的累积奖励期望值是 3/4，计算过程如下式所示：
$$
\frac{6 \times 1 + 2 \times 0}{8}=\frac{6}{8}=\frac{3}{4}
$$
**但 $s_a$ 期望的奖励到底应该是多少呢？**这边其实有两个可能的答案：一个是 0，一个是 3/4。为什么有两个可能的答案呢？这取决于你用 MC 还是TD。用 MC 跟用 TD 算出来的结果是不一样的。

假如用 MC 的话，你会发现这个 $s_a$ 就出现一次，看到 $s_a$ 这个状态，接下来累积奖励就是 0，所以 $s_a$ 期望奖励就是 0。

但 TD 在计算的时候，它要更新下面这个式子：
$$
V^{\pi}\left(s_{a}\right)=V^{\pi}\left(s_{b}\right)+r
$$

因为我们在状态 $s_a$ 得到奖励 r=0 以后，跳到状态 $s_b$。所以状态 $s_b$ 的奖励会等于状态 $s_b$ 的奖励加上在状态 $s_a$ 跳到状态 $s_b$ 的时候可能得到的奖励 r。而这个得到的奖励 r 的值是 0，$s_b$ 期望奖励是 3/4，那 $s_a$ 的奖励应该是 3/4。

用 MC 跟 TD 估出来的结果很有可能是不一样的。就算评论家观察到一样的训练数据，它最后估出来的结果也不一定是一样的。为什么会这样呢？你可能问说，哪一个结果比较对呢？其实就都对。

因为在第一个轨迹， $s_a$ 得到奖励 0 以后，再跳到 $s_b$ 也得到奖励 0。这边有两个可能。

* 一个可能是： $s_a$ 是一个标志性的状态，只要看到 $s_a$ 以后，$s_b$ 就会拿不到奖励，$s_a$ 可能影响了 $s_b$。如果是用 MC 的算法的话，它会把 $s_a$ 影响 $s_b$ 这件事考虑进去。所以看到 $s_a$ 以后，接下来 $s_b$ 就得不到奖励，$s_b$ 期望的奖励是 0。

* 另一个可能是：看到 $s_a$ 以后，$s_b$ 的奖励是 0 这件事只是一个巧合，并不是 $s_a$ 所造成，而是因为说 $s_b$ 有时候就是会得到奖励 0，这只是单纯运气的问题。其实平常 $s_b$ 会得到奖励期望值是 3/4，跟 $s_a$ 是完全没有关系的。所以假设 $s_a$ 之后会跳到 $s_b$，那其实得到的奖励按照 TD 来算应该是 3/4。

**所以不同的方法考虑了不同的假设，运算结果不同。**

## State-action Value Function(Q-function)

还有另外一种评论家叫做 `Q-function`。它又叫做`state-action value function(状态-动作价值函数)`。

* 状态价值函数的输入是一个状态，它是根据状态去计算出，看到这个状态以后的期望的累积奖励( expected accumulated reward)是多少。
* 状态-动作价值函数的输入是一个状态、动作对，它的意思是说，在某一个状态采取某一个动作，假设我们都使用演员 $\pi$ ，得到的累积奖励的期望值有多大。

Q-function 有一个需要注意的问题是，这个演员 $\pi$，在看到状态 s 的时候，它采取的动作不一定是 a。Q-function 假设在状态 s 强制采取动作 a。不管你现在考虑的这个演员 $\pi$， 它会不会采取动作 a，这不重要。在状态 s 强制采取动作 a。接下来都用演员 $\pi$ 继续玩下去，就只有在状态 s，我们才强制一定要采取动作 a，接下来就进入自动模式，让演员 $\pi$ 继续玩下去，得到的期望奖励才是 $Q^{\pi}(s,a)$ 。

![](img/6.7.png ':size=550')

Q-function 有两种写法：

* 输入是状态跟动作，输出就是一个标量；
* 输入是一个状态，输出就是好几个值。

假设动作是离散的，动作就只有 3 个可能：往左往右或是开火。那这个 Q-function 输出的 3 个值就分别代表 a 是向左的时候的 Q 值，a 是向右的时候的 Q 值，还有 a 是开火的时候的 Q 值。

要注意的事情是，上图右边的函数只有离散动作才能够使用。如果动作是无法穷举的，你只能够用上图左边这个式子，不能够用右边这个式子。

![](img/6.8.png ':size=550')

上图是文献上的结果，你去估计 Q-function 的话，看到的结果可能如上图所示。假设我们有 3 个动作：原地不动、向上、向下。

* 假设是在第一个状态，不管是采取哪个动作，最后到游戏结束的时候，得到的期望奖励其实都差不多。因为球在这个地方，就算是你向下，接下来你应该还可以急救。所以不管采取哪个动作，都差不了太多。

* 假设在第二个状态，这个乒乓球它已经反弹到很接近边缘的地方，这个时候你采取向上，你才能得到正的奖励，才接的到球。如果你是站在原地不动或向下的话，接下来你都会错过这个球。你得到的奖励就会是负的。

* 假设在第三个状态，球很近了，所以就要向上。

* 假设在第四个状态，球被反弹回去，这时候采取哪个动作就都没有差了。

这是状态-动作价值的一个例子。

![](img/6.9.png ':size=550')

虽然表面上我们学习一个 Q-function，它只能拿来评估某一个演员$\pi$ 的好坏，但只要有了这个 Q-function，我们就可以做强化学习。有了这个 Q-function，我们就可以决定要采取哪一个动作，我们就可以进行`策略改进(Policy Improvement)`。

它的大原则是这样，假设你有一个初始的演员，也许一开始很烂，随机的也没有关系。初始的演员叫做 $\pi$，这个 $\pi$ 跟环境互动，会收集数据。接下来你学习一个 $\pi$ 这个演员的 Q 值，你去衡量一下 $\pi$ 在某一个状态强制采取某一个动作，接下来用 $\pi$ 这个策略 会得到的期望奖励，用 TD 或 MC 都是可以的。你学习出一个 Q-function 以后，就保证你可以找到一个新的策略 $\pi'$ ，policy $\pi'$ 一定会比原来的策略 $\pi$ 还要好。那等一下会定义说，什么叫做好。所以假设你有一个 Q-function 和某一个策略 $\pi$，你根据策略 $\pi$ 学习出策略 $\pi$ 的 Q-function，接下来保证你可以找到一个新的策略  $\pi'$ ，它一定会比 $\pi$ 还要好，然后你用 $\pi'$ 取代 $\pi$，再去找它的 Q-function，得到新的以后，再去找一个更好的策略。**这样一直循环下去，policy 就会越来越好。** 

首先要定义的是什么叫做比较好？我们说 $\pi'$ 一定会比 $\pi$ 还要好，这边好是说，对所有可能的状态 s 而言，$V^{\pi^{\prime}}(s) \geq V^{\pi}(s)$。也就是说我们走到同一个状态 s 的时候，如果拿 $\pi$ 继续跟环境互动下去，我们得到的奖励一定会小于等于用 $\pi'$ 跟环境互动下去得到的奖励。所以不管在哪一个状态，你用 $\pi'$ 去做交互，得到的期望奖励一定会比较大。所以 $\pi'$ 是比 $\pi$  还要好的一个策略。

有了 Q-function 以后，怎么找这个 $\pi'$ 呢？如果你根据以下的这个式子去决定你的动作，
$$
\pi^{\prime}(s)=\arg \max _{a} Q^{\pi}(s, a)
$$

根据上式去决定你的动作的步骤叫做 $\pi'$ 的话，那 $\pi'$ 一定会比 $\pi$ 还要好。假设你已经学习出 $\pi$ 的 Q-function，今天在某一个状态 s，你把所有可能的动作 a 都一一带入这个 Q-function，看看哪一个 a 可以让 Q-function 的值最大，那这个动作就是 $\pi'$ 会采取的动作。

这边要注意一下，给定这个状态 s，你的策略 $\pi$  并不一定会采取动作a，我们是给定某一个状态 s 强制采取动作 a，用 $\pi$  继续互动下去得到的期望奖励，这个才是 Q-function 的定义。所以在状态 s 里面不一定会采取动作 a。用 $\pi'$ 在状态 s 采取动作 a 跟 $\pi$ 采取的动作是不一定会一样的，$\pi'$ 所采取的动作会让它得到比较大的奖励。

* 所以这个 $\pi'$ 是用 Q-function 推出来的，没有另外一个网络决定 $\pi'$ 怎么交互，有 Q-function 就可以找出 $\pi'$。
* 但是这边有另外一个问题就是，在这边要解一个 arg max 的问题，所以 a 如果是连续的就会有问题。如果是离散的，a 只有 3  个选项，一个一个带进去， 看谁的 Q 最大，没有问题。但如果 a 是连续的，要解 arg max 问题，你就会有问题。

**接下来讲一下为什么用 $Q^{\pi}(s,a)$ 决定出来的 $\pi'$ 一定会比 $\pi$ 好。**

假设有一个策略叫做 $\pi'$，它是由 $Q^{\pi}$ 决定的。我们要证对所有的状态 s 而言，$V^{\pi^{\prime}}(s) \geq V^{\pi}(s)$。

怎么证呢？我们先把 $V^{\pi}(s)$ 写出来：
$$
V^{\pi}(s)=Q^{\pi}(s, \pi(s))
$$
假设在状态 s follow $\pi$ 这个演员，它会采取的动作就是 $\pi(s)$，那你算出来的 $Q^{\pi}(s, \pi(s))$ 会等于 $V^{\pi}(s)$。一般而言，$Q^{\pi}(s, \pi(s))$ 不一定等于 $V^{\pi}(s)$ ，因为动作不一定是 $\pi(s)$。但如果这个动作是 $\pi(s)$ 的话，$Q^{\pi}(s, \pi(s))$ 是等于 $V^{\pi}(s)$ 的。


$Q^{\pi}(s, \pi(s))$ 还满足如下的关系：
$$
Q^{\pi}(s, \pi(s)) \le \max _{a} Q^{\pi}(s, a)
$$

因为 a 是所有动作里面可以让 Q 最大的那个动作，所以今天这一项一定会比它大。这一项就是 $Q^{\pi}(s, a)$，$a$ 就是 $\pi'(s)$。因为 $\pi'(s)$ 输出的 $a$ 就是可以让 $Q^\pi(s,a)$ 最大的那一个，所以我们得到了下面的式子：
$$
\max _{a} Q^{\pi}(s, a)=Q^{\pi}\left(s, \pi^{\prime}(s)\right)
$$

于是：
$$
V^{\pi}(s) \leq Q^{\pi}\left(s, \pi^{\prime}(s)\right)
$$
也就是说某一个状态，如果按照策略 $\pi$ 一直做下去，你得到的奖励一定会小于等于，在这个状态 s 你故意不按照 $\pi$ 所给你指示的方向，而是按照 $\pi'$ 的方向走一步，但只有第一步是按照 $\pi'$ 的方向走，只有在状态 s 这个地方，你才按照 $\pi'$ 的指示走，接下来你就按照 $\pi$ 的指示走。虽然只有一步之差， 但是从上面这个式子可知，虽然只有一步之差，但你得到的奖励一定会比完全 follow $\pi$ 得到的奖励还要大。

接下来要证下面的式子：
$$
Q^{\pi}\left(s, \pi^{\prime}(s) \right) \le V^{\pi'}(s)
$$

也就是说，只有一步之差，你会得到比较大的奖励。**但假设每步都是不一样的，每步都是 follow $\pi'$ 而不是 $\pi$ 的话，那你得到的奖励一定会更大。**如果你要用数学式把它写出来的话，你可以写成 $Q^{\pi}\left(s, \pi^{\prime}(s)\right)$ ，它的意思就是说，我们在状态 $s_t$ 采取动作 $a_t$，得到奖励 $r_{t}$，然后跳到状态 $s_{t+1}$，即如下式所示：

$$
Q^{\pi}\left(s, \pi^{\prime}(s)\right)=E\left[r_t+V^{\pi}\left(s_{t+1}\right) \mid s_{t}=s, a_{t}=\pi^{\prime}\left(s_{t}\right)\right]
$$
> 在文献上有时也会说：在状态 $s_t$ 采取动作 $a_t$ 得到奖励 $r_{t+1}$， 有人会写成 $r_t$，但意思其实都是一样的。

在状态 $s$ 按照 $\pi'$ 采取某一个动作 $a_t$ ，得到奖励 $r_{t}$，然后跳到状态 $s_{t+1}$，$V^{\pi}\left(s_{t+1}\right)$ 是状态 $s_{t+1}$ 根据 $\pi$ 这个演员所估出来的值。因为在同样的状态采取同样的动作，你得到的奖励和会跳到的状态不一定一样， 所以这边需要取一个期望值。

因为 $V^{\pi}(s) \leq Q^{\pi}\left(s, \pi^{\prime}(s)\right)$，也就是 $V^{\pi}(s_{t+1}) \leq Q^{\pi}\left(s_{t+1}, \pi^{\prime}(s_{t+1})\right)$，所以我们得到下式：
$$
\begin{array}{l}
E\left[r_{t}+V^{\pi}\left(s_{t+1}\right) | s_{t}=s, a_{t}=\pi^{\prime}\left(s_{t}\right)\right] \\
\leq E\left[r_{t}+Q^{\pi}\left(s_{t+1}, \pi^{\prime}\left(s_{t+1}\right)\right) | s_{t}=s, a_{t}=\pi^{\prime}\left(s_{t}\right)\right]
\end{array}
$$

因为 $Q^{\pi}\left(s_{t+1}, \pi^{\prime}\left(s_{t+1}\right)\right) = r_{t+1}+V^{\pi}\left(s_{t+2}\right)$，所以我们得到下式：
$$
\begin{array}{l}
E\left[r_{t}+Q^{\pi}\left(s_{t+1}, \pi^{\prime}\left(s_{t+1}\right)\right) | s_{t}=s, a_{t}=\pi^{\prime}\left(s_{t}\right)\right] \\
=E\left[r_{t}+r_{t+1}+V^{\pi}\left(s_{t+2}\right) | s_{t}=s, a_{t}=\pi^{\prime}\left(s_{t}\right)\right]
\end{array}
$$

然后你再代入 $V^{\pi}(s) \leq Q^{\pi}\left(s, \pi^{\prime}(s)\right)$，一直算到回合结束，即：
$$
\begin{aligned}
V^{\pi}(s) &\le Q^{\pi}(s,\pi'(s)) \\
&=E\left[r_{t}+V^{\pi}\left(s_{t+1}\right) | s_{t}=s, a_{t}=\pi^{\prime}\left(s_{t}\right)\right]\\
&\le E\left[r_{t}+Q^{\pi}\left(s_{t+1}, \pi^{\prime}\left(s_{t+1}\right)\right) | s_{t}=s, a_{t}=\pi^{\prime}\left(s_{t}\right)\right] \\
&=E\left[r_{t}+r_{t+1}+V^{\pi}\left(s_{t+2}\right) |s_{t}=s, a_{t}=\pi^{\prime}\left(s_{t}\right)\right]  \\
& \le E\left[r_{t}+r_{t+1}+Q^{\pi}\left(s_{t+2},\pi'(s_{t+2}\right) | s_{t}=s, a_{t}=\pi^{\prime}\left(s_{t}\right)\right] \\
& = E\left[r_{t}+r_{t+1}+r_{t+2}+V^{\pi}\left(s_{t+3}\right) |s_{t}=s, a_{t}=\pi^{\prime}\left(s_{t}\right)\right] \\
& \le \cdots\\
& \le E\left[r_{t}+r_{t+1}+r_{t+2}+\cdots | s_{t}=s, a_{t}=\pi^{\prime}\left(s_{t}\right)\right]  \\
& = V^{\pi'}(s)
\end{aligned}
$$


因此：
$$
V^{\pi}(s)\le V^{\pi'}(s)
$$

**从这边我们可以知道，你可以估计某一个策略的 Q-function，接下来你就可以找到另外一个策略 $\pi'$ 比原来的策略还要更好。**

## Target Network

![](img/6.12.png ':size=550')

接下来讲一下在 DQN 里一定会用到的 tip。第一个是 `目标网络(target network)`，什么意思呢？我们在学习 Q-function 的时候，也会用到 TD 的概念。那怎么用 TD？你现在收集到一个数据， 是说在状态 $s_t$，你采取动作 $a_t$ 以后，你得到奖励 $r_t$ ，然后跳到状态 $s_{t+1}$。然后根据这个 Q-function，你会知道说
$$
\mathrm{Q}^{\pi}\left(s_{t}, a_{t}\right) 
=r_{t}+\mathrm{Q}^{\pi}\left(s_{t+1}, \pi\left(s_{t+1}\right)\right)
$$

所以在学习的时候，你会说我们有 Q-function，输入 $s_t$, $a_t$ 得到的值，跟输入 $s_{t+1}$, $\pi (s_{t+1})$ 得到的值中间，我们希望它差了一个 $r_t$， 这跟刚才讲的 TD 的概念是一样的。

但是实际上这样的一个输入并不好学习，因为假设这是一个回归问题，$\mathrm{Q}^{\pi}\left(s_{t}, a_{t}\right) $ 是网络的输出，$r_{t}+\mathrm{Q}^{\pi}\left(s_{t+1}, \pi\left(s_{t+1}\right)\right)$ 是目标，你会发现目标是会动的。当然你要实现这样的训练，其实也没有问题，就是你在做反向传播的时候， $Q^{\pi}$ 的参数会被更新，你会把两个更新的结果加在一起。因为它们是同一个模型 $Q^{\pi}$， 所以两个更新的结果会加在一起。但这样会导致训练变得不太稳定，因为假设你把 $\mathrm{Q}^{\pi}\left(s_{t}, a_{t}\right) $ 当作你模型的输出，$r_{t}+\mathrm{Q}^{\pi}\left(s_{t+1}, \pi\left(s_{t+1}\right)\right)$ 当作目标的话，你要去拟合的目标是一直在变的，这种一直在变的目标的训练是不太好训练的。

所以你会把其中一个 Q 网络，通常是你会把右边这个 Q 网络固定住。也就是说你在训练的时候，你只更新左边的 Q 网络的参数，而右边的 Q 网络的参数会被固定住。因为右边的 Q 网络负责产生目标，所以叫 `目标网络`。因为目标网络是固定的，所以你现在得到的目标 $r_{t}+\mathrm{Q}^{\pi}\left(s_{t+1}, \pi\left(s_{t+1}\right)\right)$ 的值也是固定的。因为目标网络是固定的，我们只调左边网络的参数，它就变成是一个回归问题。我们希望模型的输出的值跟目标越接近越好，你会最小化它的均方误差(mean square error)。

在实现的时候，你会把左边的 Q 网络更新好几次以后，再去用更新过的 Q 网络替换这个目标网络。但它们两个不要一起动，它们两个一起动的话，结果会很容易坏掉。

一开始这两个网络是一样的，然后在训练的时候，你会把右边的 Q 网络固定住。你在做梯度下降的时候，只调左边这个网络的参数，那你可能更新 100 次以后才把这个参数复制到右边的网络去，把它盖过去。把它盖过去以后，你这个目标值就变了。就好像说你本来在做一个回归问题，那你训练 后把这个回归问题的 loss 压下去以后，接下来你把这边的参数把它复制过去以后，你的目标就变掉了，接下来就要重新再训练。

###  Intuition

![](img/6.13.png ':size=550')

我们可以通过猫追老鼠的例子来直观地理解为什么要 fix target network。猫是 `Q estimation`，老鼠是 `Q target`。一开始的话，猫离老鼠很远，所以我们想让这个猫追上老鼠。

![](img/6.14.png ':size=550')

因为 Q target 也是跟模型参数相关的，所以每次优化后，Q target 也会动。这就导致一个问题，猫和老鼠都在动。

![](img/6.15.png ':size=550')

然后它们就会在优化空间里面到处乱动，就会产生非常奇怪的优化轨迹，这就使得训练过程十分不稳定。所以我们可以固定 Q target，让老鼠动得不是那么频繁，可能让它每 5 步动一次，猫则是每一步都在动。如果老鼠每 5 次动一步的话，猫就有足够的时间来接近老鼠。然后它们之间的距离会随着优化过程越来越小，最后它们就可以拟合，拟合过后就可以得到一个最好的Q 网络。


## Exploration

![](img/6.16.png ':size=550')

**第二个 tip 是`探索(Exploration)`。**当我们使用 Q-function 的时候，policy 完全取决于 Q-function。给定某一个状态，你就穷举所有的 a， 看哪个 a 可以让 Q 值最大，它就是采取的动作。这个跟策略梯度不一样，在做策略梯度的时候，输出其实是随机的。我们输出一个动作的分布，根据这个动作的分布去做采样， 所以在策略梯度里面，你每次采取的动作是不一样的，是有随机性的。

像这种 Q-function， 如果你采取的动作总是固定的，会有什么问题呢？你会遇到的问题就是这不是一个好的收集数据的方式。因为假设我们今天真的要估某一个状态，你可以采取动作 $a_{1}$, $a_{2}$, $a_{3}$。你要估测在某一个状态采取某一个动作会得到的 Q 值，你一定要在那一个状态采取过那一个动作，才估得出它的值。如果你没有在那个状态采取过那个动作，你其实估不出那个值的。如果是用深的网络，就你的 Q-function 是一个网络，这种情形可能会没有那么严重。但是一般而言，假设 Q-function 是一个表格，没有看过的 state-action pair，它就是估不出值来。网络也是会有一样的问题，只是没有那么严重。所以今天假设你在某一个状态，动作 $a_{1}$, $a_{2}$, $a_{3}$ 你都没有采取过，那你估出来的 $Q(s,a_{1})$, $Q(s,a_{2})$, $Q(s,a_{3})$ 的值可能都是一样的，就都是一个初始值，比如说 0，即
$$
\begin{array}{l}
Q(s, a_1)=0 \\
Q(s, a_2)=0 \\
Q(s, a_3)=0
\end{array}
$$

但是假设你在状态 s，你采样过某一个动作 $a_{2}$ ，它得到的值是正的奖励。那 $Q(s, a_2)$ 就会比其他的动作都要好。在采取动作的时候， 就看说谁的 Q 值最大就采取谁，所以之后你永远都只会采样到 $a_{2}$，其他的动作就再也不会被做了，所以就会有问题。就好像说你进去一个餐厅吃饭，其实你都很难选。你今天点了某一个东西以后，假说点了某一样东西， 比如说椒麻鸡，你觉得还可以。接下来你每次去就都会点椒麻鸡，再也不会点别的东西了，那你就不知道说别的东西是不是会比椒麻鸡好吃，这个是一样的问题。

如果你没有好的探索的话，你在训练的时候就会遇到这种问题。举个例子， 假设你用 DQN 来玩`slither.io`。 你会有一个蛇，它在环境里面就走来走去，吃到星星，它就加分。假设这个游戏一开始，它往上走，然后就吃到那个星星，它就得到分数，它就知道说往上走可以得到奖励。接下来它就再也不会采取往上走以外的动作了，所以接下来就会变成每次游戏一开始，它就往上冲，然后就死掉。所以需要有探索的机制，让机器知道说，虽然根据之前采样的结果，$a_2$ 好像是不错的，但你至少偶尔也试一下 $a_{1}$ 跟 $a_{3}$，说不定它们更好。

这个问题其实就是`探索-利用窘境(Exploration-Exploitation dilemma)`问题。

有两个方法解这个问题，一个是 `Epsilon Greedy`。Epsilon Greedy($\varepsilon\text{-greedy}$) 的意思是说，我们有 $1-\varepsilon$ 的概率会按照 Q-function 来决定 动作，通常 $\varepsilon$ 就设一个很小的值， $1-\varepsilon$ 可能是 90%，也就是 90% 的概率会按照 Q-function 来决定 动作，但是你有 10% 的机率是随机的。通常在实现上 $\varepsilon$ 会随着时间递减。在最开始的时候。因为还不知道那个动作是比较好的，所以你会花比较大的力气在做探索。接下来随着训练的次数越来越多。已经比较确定说哪一个 Q 是比较好的。你就会减少你的探索，你会把 $\varepsilon$ 的值变小，主要根据 Q-function 来决定你的动作，比较少随机决定动作，这是 Epsilon Greedy。

还有一个方法叫做 `Boltzmann Exploration`，这个方法就比较像是策略梯度。在策略梯度里面，网络的输出是一个期望的动作空间上面的一个的概率分布，再根据概率分布去做采样。那其实你也可以根据 Q 值 去定一个概率分布，假设某一个动作的 Q 值越大，代表它越好，我们采取这个动作的机率就越高。但是某一个动作的 Q 值小，不代表我们不能尝试。

Q: 我们有时候也要尝试那些 Q 值比较差的动作，怎么做呢？

A: 因为 Q 值是有正有负的，所以可以它弄成一个概率，你先取指数，再做归一化。然后把 $\exp(Q(s,a))$ 做归一化的这个概率当作是你在决定动作的时候采样的概率。在实现上，Q 是一个网络，所以你有点难知道， 在一开始的时候网络的输出到底会长怎么样子。假设你一开始没有任何的训练数据，参数是随机的，那给定某一个状态 s，不同的 a 输出的值可能就是差不多的，所以一开始 $Q(s,a)$ 应该会倾向于是均匀的。也就是在一开始的时候，你这个概率分布算出来，它可能是比较均匀的。

## Experience Replay

![](img/6.17.png ':size=550')

**第三个 tip 是 `Experience Replay(经验回放)`。** Experience Replay 会构建一个 `Replay Buffer`，Replay Buffer 又被称为 `Replay Memory`。Replay Buffer 是说现在会有某一个策略$\pi$ 去跟环境做互动，然后它会去收集数据。我们会把所有的数据放到一个 buffer 里面，buffer 里面就存了很多数据。比如说 buffer 是 5 万，这样它里面可以存 5 万笔资料，每一笔资料就是记得说，我们之前在某一个状态 $s_t$，采取某一个动作 $a_t$，得到了奖励 $r_t$。然后跳到状态 $s_{t+1}$。那你用 $\pi$ 去跟环境互动很多次，把收集到的资料都放到这个 replay buffer 里面。

这边要注意是 replay buffer 里面的经验可能是来自于不同的策略，你每次拿 $\pi$ 去跟环境互动的时候，你可能只互动 10000 次，然后接下来就更新你的 $\pi$ 了。但是这个 buffer 里面可以放 5 万笔资料，所以 5 万笔资料可能是来自于不同的策略。Buffer 只有在它装满的时候，才会把旧的资料丢掉。所以这个 buffer 里面它其实装了很多不同的策略的经验。

![](img/6.18.png ':size=550')

有了 buffer 以后，怎么训练 Q 的模型，怎么估 Q-function？你的做法是这样：迭代地去训练这个 Q-function，在每次迭代里面，从这个 buffer 里面随机挑一个 batch 出来，就跟一般的网络训练一样，从那个训练集里面，去挑一个 batch 出来。你去采样一个 batch 出来，里面有一把的经验，根据这把经验去更新你的 Q-function。就跟 TD learning 要有一个目标网络是一样的。你去采样一堆 batch，采样一个 batch 的数据，采样一堆经验，然后再去更新你的 Q-function。

当我们这么做的时候， 它变成了一个 `off-policy` 的做法。因为本来我们的 Q 是要观察 $\pi$ 的经验，但实际上 replay buffer 里面的这些经验不是通通来自于 $\pi$，有些是过去其他的 $\pi$  所遗留下来的经验。因为你不会拿某一个 $\pi$ 就把整个 buffer 装满，然后拿去测 Q-function，这个 $\pi$ 只是采样一些数据塞到那个 buffer 里面去，然后接下来就让 Q 去训练。所以 Q 在采样的时候， 它会采样到过去的一些资料。

这么做有两个好处：

* 其实在做强化学习的时候， 往往最花时间的步骤是在跟环境做互动，训练网络反而是比较快的。因为用 GPU 训练其实很快， 真正花时间的往往是在跟环境做互动。用 replay buffer 可以减少跟环境做互动的次数，因为在做训练的时候，你的经验不需要通通来自于某一个策略。一些过去的策略所得到的经验可以放在 buffer 里面被使用很多次，被反复的再利用，这样让采样到经验的利用是比较高效的。

* 在训练网络的时候，其实我们希望一个 batch 里面的数据越多样(diverse)越好。如果 batch 里面的数据都是同样性质的，训练下去是容易坏掉的。如果 batch 里面都是一样的数据，训练的时候，performance 会比较差。我们希望 batch 的数据越多样越好。那如果 buffer 里面的那些经验通通来自于不同的策略，那采样到的一个 batch 里面的数据会是比较多样的。

Q：我们明明是要观察 $\pi$ 的值，里面混杂了一些不是 $\pi$ 的经验，这有没有关系？

A：没关系。这并不是因为过去的 $\pi$ 跟现在的 $\pi$ 很像， 就算过去的 $\pi$ 没有很像，其实也是没有关系的。主要的原因是因为， 我们并不是去采样一个轨迹，我们只采样了一笔经验，所以跟是不是 off-policy 这件事是没有关系的。就算是 off-policy，就算是这些经验不是来自于 $\pi$，我们其实还是可以拿这些经验来估测 $Q^{\pi}(s,a)$。这件事有点难解释，不过你就记得说 Experience Replay 在理论上也是没有问题的。

## DQN

![](img/dqn.png ':size=550')

DQN 使用深度卷积神经网络近似拟合状态动作值函数 $Q(s,a)$，其网络结构如上图所示。DQN 模型的输入是距离当前时刻最近的 4 帧图像，该输入经过 3 个卷积层和 2 个全连接层的非线性变化后，最终在输出层输出每个动作对应的 Q 值。

![](img/6.19.png ':size=550')


上图就是一般的 `Deep Q-network(DQN)` 的算法。

这个算法是这样的。初始化的时候，你初始化 2 个网络：Q 和 $\hat{Q}$，其实 $\hat{Q}$ 就等于 Q。一开始这个目标 Q 网络，跟你原来的 Q 网络是一样的。在每一个 episode，你拿你的演员去跟环境做互动，在每一次互动的过程中，你都会得到一个状态 $s_t$，那你会采取某一个动作 $a_t$。怎么知道采取哪一个动作 $a_t$ 呢？你就根据你现在的 Q-function。但是你要有探索的机制。比如说你用 Boltzmann 探索或是 Epsilon Greedy 的探索。那接下来你得到奖励 $r_t$，然后跳到状态 $s_{t+1}$。所以现在收集到一笔数据，这笔数据是 ($s_t$, $a_t$ ,$r_t$, $s_{t+1}$)。这笔数据就塞到你的 buffer 里面去。如果 buffer 满的话， 你就再把一些旧的资料丢掉。接下来你就从你的 buffer 里面去采样数据，那你采样到的是 $(s_{i}, a_{i}, r_{i}, s_{i+1})$。这笔数据跟你刚放进去的不一定是同一笔，你可能抽到一个旧的。要注意的是，其实你采样出来不是一笔数据，你采样出来的是一个 batch 的数据，你采样一个 batch 出来，采样一把经验出来。接下来就是计算你的目标。假设采样出这么一笔数据。根据这笔数据去算你的目标。你的目标是什么呢？目标记得要用目标网络 $\hat{Q}$ 来算。目标是：

$$
y=r_{i}+\max _{a} \hat{Q}\left(s_{i+1}, a\right)
$$
其中 a 就是让 $\hat{Q}$ 的值最大的 a。因为我们在状态 $s_{i+1}$会采取的动作 a，其实就是那个可以让 Q 值最大的那一个 a。接下来我们要更新 Q 的值，那就把它当作一个回归问题。希望 $Q(s_i,a_i)$  跟你的目标越接近越好。然后假设已经更新了某一个数量的次，比如说 C 次，设 C = 100， 那你就把 $\hat{Q}$ 设成 Q，这就是 DQN。

Q: DQN 和 Q-learning 有什么不同？

A: 整体来说，DQN 与 Q-learning 的目标价值以及价值的更新方式都非常相似，主要的不同点在于：

* DQN 将 Q-learning 与深度学习结合，用深度网络来近似动作价值函数，而 Q-learning 则是采用表格存储；
* DQN 采用了经验回放的训练方法，从历史数据中随机采样，而 Q-learning 直接采用下一个状态的数据进行学习。

## 一些技巧

下面我们介绍下 DQN 的基本技巧：

* 在 Atari 游戏里面，一般 mini-batch 设置为 32。
* Experience replay 用在新问题上一般为 $10^6$。

# Tips of Q-learning
## Double DQN
![](img/7.1.png)

接下来要讲的是训练 Q-learning 的一些 tips。第一个 tip 是做 `Double DQN`。为什么要有 Double DQN 呢？因为在实现上，你会发现 Q 值往往是被高估的。上图来自于 Double DQN 的原始 paper，它想要显示的结果就是 Q 值往往是被高估的。

这边有 4 个不同的小游戏，横轴是训练的时间，红色锯齿状一直在变的线就是 Q-function 对不同的状态估计出来的平均 Q 值，有很多不同的状态，每个状态你都 sample 一下，然后算它们的 Q 值，把它们平均起来。

这条红色锯齿状的线在训练的过程中会改变，但它是不断上升的。因为 Q-function 是取决于你的策略的。学习的过程中你的策略越来越强，你得到的 Q 值会越来越大。在同一个状态， 你得到 reward 的期望会越来越大，所以一般而言，这个值都是上升的，但这是 Q-network 估测出来的值。

接下来你真地去算它，怎么真地去算？你有策略，然后真的去玩那个游戏，就玩很多次，玩个一百万次。然后就去真地算说，在某一个状态， 你会得到的 Q 值到底有多少。你会得到在某一个状态采取某一个动作。你接下来会得到累积奖励(accumulated reward)是多少。你会发现估测出来的值远比实际的值大，在每一个游戏都是这样，都大很多。所以今天要提出 Double DQN 的方法，它可以让估测的值跟实际的值是比较接近的。

我们先看它的结果，蓝色的锯齿状的线是 Double DQN 的 Q-network 所估测出来的 Q 值，蓝色的无锯齿状的线是真正的 Q 值，你会发现它们是比较接近的。 用网络估测出来的就不用管它，比较没有参考价值。用 Double DQN 得出来真正的累积奖励，在这 3 种情况下都是比原来的 DQN 高的，代表 Double DQN 学习出来的那个策略比较强。所以它实际上得到的 reward 是比较大的。虽然一般的 DQN 的 Q-network 高估了自己会得到的 reward，但实际上它得到的 reward 是比较低的。

![](img/7.2.png)

Q: 为什么 Q 值总是被高估了呢？

A:因为实际上在做的时候，是要让左边这个式子跟右边这个目标越接近越好。你会发现目标的值很容易一不小心就被设得太高。因为在算这个目标的时候，我们实际上在做的事情是，看哪一个 a 可以得到最大的 Q 值，就把它加上去，就变成我们的目标。所以假设有某一个动作得到的值是被高估的。

举例来说， 现在有 4 个动作，本来它们得到的值都是差不多的，它们得到的 reward 都是差不多的。但是在估计的时候，网络是有误差的。

* 假设是第一个动作被高估了，假设绿色的东西代表是被高估的量，它被高估了，那这个目标就会选这个动作，然后就会选这个高估的 Q 值来加上 $r_t$，来当作你的目标。
* 如果第四个动作被高估了，那就会选第四个动作来加上 $r_t$ 来当作你的目标值。所以你总是会选那个 Q 值被高估的，你总是会选那个 reward 被高估的动作当作这个 max 的结果去加上 $r_t$ 当作你的目标，所以你的目标总是太大。

![](img/7.3.png)
Q: 怎么解决目标值总是太大的问题呢？

A: 在 Double DQN 里面，选动作的 Q-function 跟算值的 Q-function 不是同一个。在原来的 DQN 里面，你穷举所有的 a，把每一个 a 都带进去， 看哪一个 a 可以给你的 Q 值最高，那你就把那个 Q 值加上 $r_t$。但是在 Double DQN 里面，你有两个 Q-network：

* 第一个 Q-network Q 决定哪一个动作的 Q 值最大（你把所有的 a 带入 Q 中，看看哪一个 Q 值最大）。
* 你决定你的动作以后，你的 Q 值是用 $Q'$ 算出来的。

假设我们有两个 Q-function，

* 假设第一个 Q-function 高估了它现在选出来的动作 a，只要第二个 Q-function $Q'$ 没有高估这个动作 a 的值，那你算出来的就还是正常的值。
* 假设 $Q'$ 高估了某一个动作的值，那也没差，因为只要前面这个 Q 不要选那个动作出来就没事了，这个就是 Double DQN 神奇的地方。

Q: 哪来 Q  跟 $Q'$ 呢？哪来两个网络呢？

A: 在实现上，你有两个 Q-network：目标的 Q-network 和你会更新的 Q-network。所以在 Double DQN 里面，你会拿你会更新参数的那个 Q-network 去选动作，然后你拿目标网络（固定住不动的网络）去算值。

Double DQN 相较于原来的 DQN 的更改是最少的，它几乎没有增加任何的运算量，连新的网络都不用，因为原来就有两个网络了。你唯一要做的事情只有，本来你在找 Q 值最大的 a 的时候，你是用 $Q'$ 来算，你是用目标网络来算，现在改成用另外一个会更新的 Q-network 来算。

假如你今天只选一个 tip 的话，正常人都是实现 Double DQN，因为很容易实现。

## Dueling DQN
![](img/7.4.png)
第二个 tip 是 `Dueling DQN`。其实 Dueling DQN 也蛮好做的，相较于原来的 DQN，它唯一的差别是改了网络的架构。Q-network 就是输入状态，输出就是每一个动作的 Q 值。Dueling DQN 唯一做的事情是改了网络的架构，其它的算法都不需要动。

Q: Dueling DQN 是怎么改了网络的架构呢？

A: 本来的 DQN 就是直接输出 Q 值的值。现在这个 dueling 的 DQN，就是下面这个网络的架构。它不直接输出 Q 值的值，它分成两条路径去运算：

* 第一条路径会输出一个 scalar，这个 scalar 叫做 $V(s)$。因为它跟输入 s 是有关系，所以叫做 $V(s)$，$V(s)$ 是一个 scalar。
* 第二条路径会输出一个 vector，这个 vector 叫做 $A(s,a)$。下面这个 vector，它是每一个动作都有一个值。

你再把这两个东西加起来就可以得到你的 Q 值。

![](img/7.5.png)

Q: 这么改有什么好处？

A : 那我们假设说，原来的 $Q(s,a)$ 就是一个表格。我们假设状态是离散的，实际上状态不是离散的。为了说明方便，我们假设就是只有 4 个不同的状态，只有 3 个不同的动作，所以 $Q(s,a)$  你可以看作是一个表格。

我们知道：
$$
Q(s,a) = V(s) + A(s,a)
$$

其中

* $V(s)$ 是对不同的状态 它都有一个值。 
* $A(s,a)$ 它是对不同的状态，不同的动作都有一个值。

你把这个 V 的值加到 A 的每一列就会得到 Q 的值。把 2+1，2+(-1)，2+0，就得到 3，1，2，以此类推。

如上图所示，假设说你在训练网络的时候，目标是希望这一个值变成 4，这一个值变成 0。但是你实际上能更改的并不是 Q 的值，你的网络更改的是 V 跟 A 的值。根据网络的参数，V 跟 A 的值输出以后，就直接把它们加起来，所以其实不是更动 Q 的值。

然后在学习网络的时候，假设你希望这边的值，这个 3 增加 1 变成 4，这个 -1 增加 1 变成 0。最后你在训练网络的时候，网络 可能会说，我们就不要动这个 A 的值，就动 V 的值，把 V 的值从 0 变成 1。把 0 变成 1 有什么好处呢？你会发现说，本来你只想动这两个东西的值，那你会发现说，这个第三个值也动了，-2 变成  -1。所以有可能说你在某一个状态，你明明只 sample 到这 2 个动作，你没 sample 到第三个动作，但是你其实也可以更改第三个动作的 Q 值。这样的好处就是你不需要把所有的 state-action pair 都 sample 过，你可以用比较高效的方式去估计 Q 值出来。因为有时候你更新的时候，不一定是更新下面这个表格。而是只更新了 $V(s)$，但更新 $V(s)$ 的时候，只要一改所有的值就会跟着改。这是一个比较有效率的方法，去使用你的数据，这个是 Dueling DQN 可以带给我们的好处。

那可是接下来有人就会问说会不会最后 学习出来的结果是说，反正 machine 就学到 V 永远都是 0，然后反正 A 就等于 Q，那你就没有得到任何 Dueling DQN 可以带给你的好处， 就变成跟原来的 DQN 一模一样。为了避免这个问题，实际上你要给 A 一些约束，让 更新 A 其实比较麻烦，让网络倾向于会想要去用 V 来解问题。

举例来说，你可以看原始的文献，它有不同的约束 。一个最直觉的约束是你必须要让这个 A 的每一列的和都是 0，所以看我这边举的例子，列的和都是 0。如果这边列的和都是 0，这边这个 V 的值，你就可以想成是上面 Q 的每一列的平均值。这个平均值，加上这些值才会变成是 Q 的 值。所以今天假设你发现说你在更新参数的时候，你是要让整个列一起被更新。你就不会想要更新这边，因为你不会想要更新 Ａ 这个矩阵。因为 A 这个矩阵的每一列的和都要是 0，所以你没有办法说，让这边的值，通通都 +1，这件事是做不到的。因为它的约束就是你的和永远都是要 0。所以不可以都 +1，这时候就会强迫网络去更新 V 的值，然后让你可以用比较有效率的方法，去使用你的数据。

![](img/7.6.png)

实现时，你要给这个 A 一个约束。举个例子，假设你有 3 个动作，然后在这边输出的 vector 是 $[7,3,2]^{\mathrm{T}}$，你在把这个 A 跟这个 V  加起来之前，先加一个归一化(normalization)，就好像做那个层归一化( layer normalization)一样。加一个归一化，这个归一化做的事情就是把 7+3+2 加起来等于 12，12/3 = 4。然后把这边通通减掉 4，变成 3, -1, 2。再把 3, -1, 2 加上 1.0，得到最后的 Q 值。这个归一化的步骤就是网络的其中一部分，在训练的时候，你从这边也是一路 back propagate 回来的，只是归一化是没有参数的，它只是一个归一化的操作。把它可以放到网络里面，跟网络的其他部分 jointly trained，这样 A 就会有比较大的约束。这样网络就会给它一些好处，倾向于去更新 V 的值，这个是 Dueling DQN。


## Prioritized Experience Replay

![](img/7.7.png)
有一个技巧叫做 `Prioritized Experience Replay`。Prioritized Experience Replay 是什么意思呢？

我们原来在 sample 数据去训练你的 Q-network 的时候，你是均匀地从 experience buffer 里面去 sample 数据。那这样不见得是最好的， 因为也许有一些数据比较重要。假设有一些数据，你之前有 sample 过。你发现这些数据的 TD error 特别大（TD error 就是网络的输出跟目标之间的差距），那这些数据代表说你在训练网络的时候， 你是比较训练不好的。那既然比较训练不好， 那你就应该给它比较大的概率被 sample 到，即给它 `priority`。这样在训练的时候才会多考虑那些训练不好的训练数据。实际上在做 prioritized experience replay 的时候，你不仅会更改 sampling 的 process，你还会因为更改了 sampling 的过程，更改更新参数的方法。所以 prioritized experience replay 不仅改变了 sample 数据的分布，还改变了训练过程。
## Balance between MC and TD

![](img/7.8.png)
**另外一个可以做的方法是 balance MC 跟 TD。**MC 跟 TD 的方法各自有各自的优劣，怎么在 MC 跟 TD 里面取得一个平衡呢？我们的做法是这样，在 TD 里面，在某一个状态 $s_t$ 采取某一个动作$a_t$ 得到 reward $r_t$，接下来跳到那一个状态 $s_{t+1}$。但是我们可以不要只存一个步骤的数据，我们存 N 个步骤的数据。

我们记录在 $s_t$ 采取 $a_t$，得到 $r_t$，会跳到什么样 $s_t$。一直纪录到在第 N 个步骤以后，在 $s_{t+N}$采取 $a_{t+N}$ 得到 reward $r_{t+N}$，跳到 $s_{t+N+1}$ 的这个经验，通通把它存下来。实际上你今天在做更新的时候， 在做 Q-network learning 的时候，你的 learning 的方法会是这样，你 learning 的时候，要让 $Q(s_t,a_t)$ 跟你的目标值越接近越好。$\hat{Q}$ 所计算的不是 $s_{t+1}$，而是 $s_{t+N+1}$的。你会把 N 个步骤以后的状态 丢进来，去计算 N 个步骤以后，你会得到的 reward。要算目标值的话，要再加上多步(multi-step) 的 reward $\sum_{t^{\prime}=t}^{t+N} r_{t^{\prime}}$ ，多步的 reward 是从时间 t 一直到 t+N 的 N 个 reward 的和。然后希望你的 $Q(s_t,a_t)$ 和目标值越接近越好。

你会发现说这个方法就是 MC 跟 TD 的结合。因此它就有 MC 的好处跟坏处，也有 TD 的好处跟坏处。如果看它的这个好处的话，因为我们现在 sample 了比较多的步骤，之前是只 sample 了一个步骤， 所以某一个步骤得到的数据是 real 的，接下来都是 Q 值估测出来的。现在 sample 比较多步骤，sample N 个步骤才估测值，所以估测的部分所造成的影响就会比小。当然它的坏处就跟 MC 的坏处一样，因为你的 r 比较多项，你把 N 项的 r 加起来，方差就会比较大。但是你可以去调这个 N 的值，去在 方差 跟不精确的 Q 之间取得一个平衡。N 就是一个 hyperparameter，你要调这个 N 到底是多少，你是要多 sample 三步，还是多 sample 五步。

## Noisy Net
![](img/7.9.png)
我们还可以改进探索。Epsilon Greedy 这样的探索是在动作的空间上面加噪声，**但是有一个更好的方法叫做`Noisy Net`，它是在参数的空间上面加噪声。**

Noisy Net 的意思是说，每一次在一个 episode 开始的时候，在你要跟环境互动的时候，你就把你的 Q-function 拿出来，Q-function 里面其实就是一个网络，就变成你把那个网络拿出来，在网络的每一个参数上面加上一个高斯噪声(Gaussian noise)，那你就把原来的 Q-function 变成 $\tilde{Q}$ 。因为 $\hat{Q}$ 已经用过，$\hat{Q}$ 是那个目标网络，我们用 $\tilde{Q}$  来代表一个 `Noisy Q-function`。我们把每一个参数都加上一个高斯噪声，就得到一个新的网络叫做 $\tilde{Q}$。

这边要注意在每个 episode 开始的时候，开始跟环境互动之前，我们就 sample 网络。接下来你就会用这个固定住的 noisy网络去玩这个游戏，直到游戏结束，你才重新再去 sample 新的噪声。OpenAI  跟 DeepMind 又在同时间提出了一模一样的方法，通通都发表在 ICLR 2018，两篇 paper 的方法就是一样的。不一样的地方是，他们用不同的方法，去加噪声。OpenAI 加的方法好像比较简单，他就直接加一个高斯噪声就结束了，就你把每一个参数，每一个 weight 都加一个高斯噪声就结束了。DeepMind 做比较复杂，他们的噪声是由一组参数控制的，也就是说网络可以自己决定说它那个噪声要加多大，但是概念就是一样的。总之就是把你的 Q-function 的里面的那个网络加上一些噪声，把它变得有点不一样，跟原来的 Q-function 不一样，然后拿去跟环境做互动。两篇 paper 里面都有强调说，你这个参数虽然会加噪声，但在同一个 episode 里面你的参数就是固定的，你是在换 episode， 玩第二场新的游戏的时候，你才会重新 sample 噪声，在同一场游戏里面就是同一个 noisy Q-network 在玩那一场游戏，这件事非常重要。为什么这件事非常重要呢？因为这是导致了 Noisy Net 跟原来的 Epsilon Greedy 或其它在动作做 sample 方法的本质上的差异。

![](img/7.10.png)

有什么样本质上的差异呢？在原来 sample 的方法，比如说 Epsilon Greedy 里面，就算是给同样的状态，你的 agent 采取的动作也不一定是一样的。因为你是用 sample 决定的，给定同一个状态，要根据 Q-function 的网络，你会得到一个动作，你 sample 到 random，你会采取另外一个 动作。所以给定同样的状态，如果你今天是用 Epsilon Greedy 的方法，它得到的动作是不一样的。但实际上你的策略并不是这样运作的啊。在一个真实世界的策略，给同样的状态，他应该会有同样的回应。而不是给同样的状态，它其实有时候吃 Q-function，然后有时候又是随机的，所以这是一个不正常的动作，是在真实的情况下不会出现的动作。但是如果你是在 Q-function 上面去加噪声的话， 就不会有这个情形。因为如果你今天在 Q-function 上加噪声，在 Q-function 的网络的参数上加噪声，那在整个互动的过程中，在同一个 episode 里面，它的网络的参数总是固定的，所以看到同样的状态，或是相似的状态，就会采取同样的动作，那这个是比较正常的。在 paper 里面有说，这个叫做 `state-dependent exploration`，也就是说你虽然会做探索这件事， 但是你的探索是跟状态有关系的，看到同样的状态， 你就会采取同样的探索的方式，而 noisy 的动作只是随机乱试。但如果你是在参数下加噪声，那在同一个 episode 里面，里面你的参数是固定的。那你就是有系统地在尝试，每次会试说，在某一个状态，我都向左试试看。然后再下一次在玩这个同样游戏的时候，看到同样的状态，你就说我再向右试试看，你是有系统地在探索这个环境。

## Distributional Q-function
![](img/7.11.png)

还有一个技巧叫做 `Distributional Q-function`。我们不讲它的细节，只告诉你大致的概念。Distributional Q-function 还蛮有道理的， 但是它没有红起来。你就发现说没有太多人真的在实现的时候用这个技术，可能一个原因就是它不好实现。Q-function 是累积奖励的期望值，所以我们算出来的这个 Q 值其实是一个期望值。因为环境是有随机性的，在某一个状态 采取某一个动作的时候，我们把所有的 reward 玩到游戏结束的时候所有的 reward 进行一个统计，你其实得到的是一个分布。也许在 reward 得到 0 的机率很高，在 -10 的概率比较低，在 +10 的概率比较低，但是它是一个分布。我们对这一个分布算它的平均值才是这个 Q 值，我们算出来是累积奖励的期望。所以累积奖励是一个分布，对它取期望，对它取平均值，你得到了 Q 值。但不同的分布，它们其实可以有同样的平均值。也许真正的分布是右边的分布，它算出来的平均值跟左边的分布 算出来的平均值 其实是一样的，但它们背后所代表的分布其实是不一样的。假设我们只用 Q 值的期望来代表整个 reward 的话，其实可能会丢失一些信息，你没有办法 model reward 的分布。

![](img/7.12.png)

Distributional Q-function 它想要做的事情是对分布(distribution)建模，怎么做呢？在原来的 Q-function 里面，假设你只能够采取 $a_1$, $a_2$, $a_3$ 3 个动作，那你就是输入一个状态，输出 3 个值。3 个值分别代表 3 个动作的 Q 值，但是这个 Q 值是一个分布的期望值。所以 Distributional Q-function 的想法就是何不直接输出那个分布。但是要直接输出一个分布也不知道怎么做。

实际上的做法是说， 假设分布的值就分布在某一个 range 里面，比如说 -10 到 10，那把 -10 到 10 中间拆成一个一个的 bin，拆成一个一个的长条图。举例来说，在这个例子里面，每一个动作的 reward 的空间就拆成 5 个 bin。假设 reward 可以拆成 5 个 bin 的话，今天你的 Q-function 的输出是要预测说，你在某一个状态，采取某一个动作，你得到的 reward，落在某一个 bin 里面的概率。

所以其实这边的概率的和，这些绿色的 bar 的和应该是 1，它的高度代表说，在某一个状态 采取某一个动作的时候，它落在某一个 bin 的机率。这边绿色的代表动作 1，红色的代表动作 2，蓝色的代表动作 3。所以今天你就可以真的用 Q-function 去估计 $a_1$ 的分布，$a_2$ 的分布，$a_3$ 的分布。那实际上在做测试的时候， 我们还是要选某一个动作去执行，那选哪一个动作呢？实际上在做的时候，还是选这个平均值最大的那个动作去执行。

但假设我们可以对 distribution 建模的话，除了选平均值最大的以外，也许在未来你可以有更多其他的运用。举例来说，你可以考虑它的分布长什么样子。如果分布方差很大，代表说采取这个动作虽然平均值可能平均而言很不错，但也许风险很高，你可以训练一个网络它是可以规避风险的。就在 2 个动作平均值都差不多的情况下，也许可以选一个风险比较小的动作来执行，这是 Distributional Q-function 的好处。关于怎么训练这样的 Q-network 的细节，我们就不讲，你只要记得说  Q-network 有办法输出一个分布就对了。我们可以不只是估测得到的期望 reward 平均值的值。我们其实是可以估测一个分布的。

## Rainbow

![](img/7.13.png)

**最后一个技巧叫做 rainbow，把刚才所有的方法都综合起来就变成 rainbow 。**因为刚才每一个方法，就是有一种自己的颜色，把所有的颜色通通都合起来，就变成 rainbow，它把原来的 DQN 也算是一种方法，故有 7 色。

那我们来看看这些不同的方法。横轴是训练过程，纵轴是玩了 10 几个 Atari 小游戏的平均的分数的和，但它取的是中位数的分数，为什么是取中位数不是直接取平均呢？因为它说每一个小游戏的分数，其实差很多。如果你取平均的话，到时候某几个游戏就控制了你的结果，所以它取中位数的值。

如果你是一般的 DQN，就灰色这一条线，就没有很强。那如果是你换 Noisy DQN，就强很多。如果这边每一个单一颜色的线是代表说只用某一个方法，那紫色这一条线是 DDQN(Double DQN)，DDQN 还蛮有效的。然后 Prioritized DDQN、Dueling DDQN 和 Distributional DQN 都蛮强的，它们都差不多很强的。A3C 其实是 Actor-Critic 的方法。单纯的 A3C 看起来是比 DQN 强的。这边怎么没有多步的方法，多步的方法就是平衡 TD 跟 MC，我猜是因为 A3C 本身内部就有做多步的方法，所以他可能觉得说有实现 A3C 就算是有实现多步的方法。所以可以把这个 A3C 的结果想成是多步方法的结果。其实这些方法他们本身之间是没有冲突的，所以全部都用上去就变成七彩的一个方法，就叫做 rainbow，然后它很高。

![](img/7.14.png)

上图是说，在 rainbow 这个方法里面， 如果我们每次拿掉其中一个技术，到底差多少。因为现在是把所有的方法都加在一起，发现说进步很多，但会不会有些方法其实是没用的。所以看看说， 每一个方法哪些方法特别有用，哪些方法特别没用。这边的虚线就是拿掉某一种方法以后的结果，你会发现说，黄色的虚线，拿掉多步掉很多。Rainbow 是彩色这一条，拿掉多步会掉下来。拿掉 Prioritized  Experience Replay 后也马上就掉下来。拿掉分布，它也掉下来。

这边有一个有趣的地方是说，在开始的时候，分布的训练的方法跟其他方法速度差不多。但是如果你拿掉分布的时候，你的训练不会变慢，但是性能(performance)最后会收敛在比较差的地方。拿掉 Noisy Net 后性能也是差一点。拿掉 Dueling 也是差一点。拿掉 Double 没什么差，所以看来全部合在一起的时候，Double 是比较没有影响的。其实在 paper 里面有给一个 make sense 的解释，其实当你有用 Distributional DQN 的时候，本质上就不会高估你的 reward。我们是为了避免高估 reward 才加了 Double DQN。那在 paper 里面有讲说，如果有做 Distributional DQN，就比较不会有高估的结果。 事实上他有真的算了一下发现说，其实多数的状况是低估 reward 的，所以变成 Double DQN 没有用。

为什么做 Distributional DQN，不会高估 reward，反而会低估 reward 呢？因为这个 Distributional DQN 的输出的是一个分布的范围，输出的范围不可能是无限宽的，你一定是设一个范围， 比如说最大输出范围就是从 -10 到 10。假设今天得到的 reward 超过 10 怎么办？是 100 怎么办，就当作没看到这件事。所以 reward 很极端的值，很大的值其实是会被丢掉的， 所以用 Distributional DQN 的时候，你不会有高估的现象，反而会低估。

# 针对连续动作的 DQN

## 方案 1 & 方案 2
跟基于策略梯度的方法比起来，DQN 是比较稳的。策略梯度是没有太多游戏是玩得起来的，策略梯度比较不稳，在没有 近端策略优化 之前，我们很难用策略梯度做什么事情。DQN 相对而言是比较稳的。最早 DeepMind 的论文拿深度强化学习来玩雅达利的游戏，用的就是 DQN。DQN 比较容易训练的一个理由是：在 DQN 里面，你只要能够估计出Q函数，就保证你一定可以找到一个比较好的策略。也就是你只要能够估计出Q函数，就保证你可以改进策略。而估计Q函数这件事情，是比较容易的，因为它就是一个回归问题。在回归问题里面， 你可以轻易地知道模型学习得是不是越来越好，只要看那个回归的损失有没有下降，你就知道说模型学习得好不好，所以估计Q函数相较于学习一个策略是比较容易的。你只要估计Q函数，就可以保证说现在一定会得到比较好的策略。所以一般而言 DQN 比较容易操作。

DQN 其实存在一些问题，最大的问题是它不太容易处理连续动作。很多时候动作是连续的，比如我们玩雅达利的游戏，智能体只需要决定比如说上下左右，这种动作是离散的。那很多时候动作是连续的。举例来说假设智能体要做的事情是开自驾车，它要决定说它方向盘要左转几度， 右转几度，这是连续的。假设智能体是一个机器人，它身上有 50 个 关节，它的每一个动作就对应到它身上的这 50 个关节的角度。而那些角度也是连续的。所以很多时候动作并不是一个离散的东西，它是一个向量。在这个向量里面，它的每一个维度都有一个对应的值，都是实数，它是连续的。假设动作是连续的，做 DQN 就会有困难。因为在做 DQN 里面一个很重要的一步是你要能够解这个优化问题。估计出 Q函数$Q(s,a)$ 以后，必须要找到一个 $a$，它可以让 $Q(s,a)$ 最大，如下式所示。

$$
  a=\arg \max _{a} Q(s, a)
$$

假设$a$是离散的，即$a$的可能性都是有限的。举例来说，雅达利的小游戏里面，$a$ 就是上下左右跟开火，它是有限的，我们可以把每一个可能的动作都带到 Q 里面算它的 Q 值。但假如$a$是连续的，你无法穷举所有可能的连续动作，试试看哪一个连续动作可以让 Q 的值最大。

怎么解这个问题呢？就有各种不同的方案。

第一个方案是假设你不知道怎么解这个问题，因为$a$是没有办法穷举的，怎么办？我们可以采样出 $N$ 个可能的 $a$：$\left\{a_{1}, a_{2}, \cdots, a_{N}\right\}$ ，一个一个带到 Q函数里面，看谁最大。这个方法其实也不会太不高效， 因为你在运算的时候会使用 GPU，一次会把 $N$ 个连续动作都丢到 Q函数里面，一次得到 $N$ 个 Q 值，然后看谁最大。当然这不是一个非常精确的做法，因为你没有办法做太多的采样， 所以你估计出来的 Q 值，最后决定的动作可能不是非常的精确，这是第一个方案。

第二个方案是什么呢？既然要解的是一个优化问题（optimization problem），其实是要最大化目标函数（objective function），要最大化一个东西， 就可以用梯度上升。我们就把$a$当作是参数，然后要找一组$a$去最大化Q函数，就用梯度上升去更新 $a$ 的值，最后看看能不能找到一个$a$去最大化Q函数，也就是目标函数。当然这样子你会遇到全局最大值（global maximum）的问题， 就不见得能够真的找到最优的结果，而且这个运算量显然很大， 因为你要迭代地更新 $a$。我们训练一个网络就很花时间了。如果你用梯度上升的方法来处理连续的问题， 等于是你每次要决定采取哪一个动作的时候，都还要做一次训练网络的过程，显然运算量是很大的。这是第二个方案。


## 方案 3：设计网络

第三个方案是特别设计一个网络的架构，特别设计Q函数，使得解 arg max 的问题变得非常容易。也就是这边的Q函数不是一个一般的Q函数，特别设计一下它的样子，让你要找让这个Q函数最大的 $a$ 的时候非常容易。

下图是一个例子，这边有Q函数，这个Q函数的做法是这样。
  通常输入状态 $s$ 就是一个图像，可以用一个向量或一个矩阵来表示。
  输入 $s$，Q函数会输出 3 个东西。它会输出 $\mu(s)$，这是一个向量。它会输出 $\Sigma(s)$ ，这是一个矩阵。它会输出 $V(s)$，这是一个标量。
  输出这 3 个东西以后，我们知道Q函数其实是吃一个$s$跟 $a$，然后决定一个值。Q函数意思是说在某一个状态，采取某一个动作的时候，你期望的奖励有多大。到目前为止这个Q函数只吃 $s$，它还没有吃$a$进来，$a$ 在哪里呢？当这个Q函数吐出 $\mu$、 $\Sigma$ 跟 $V$ 的时候，我们才把$a$引入，用$a$跟 $\mu(s)、\Sigma(s)、V$  互相作用一下，你才算出最终的 Q 值。

![](img/8.2.png)

 $a$怎么和这 3 个东西互相作用呢？实际上 $Q(s,a)$，Q函数的运作方式是先输入 $s$，让你得到 $\mu,\Sigma$ 跟 $V$。然后再输入 $a$，然后接下来把$a$跟 $\mu$ 相减。注意一下$a$现在是连续的动作，所以它也是一个向量。假设你现在是要操作机器人的话，这个向量的每一个维度，可能就对应到机器人的某一个关节，它的数值就是关节的角度，所以$a$是一个向量。把向量 $a$ 减掉向量 $\mu$，取转置，所以它是一个横的向量。$\Sigma$ 是一个矩阵。然后$a$减掉 $\mu(s)$ ，$a$ 和 $\mu(s)$ 都是向量，减掉以后还是一个竖的向量。所以 $-(a-\mu(s))^{T} \Sigma(s)(a-\mu(s))+V(s)$ 是一个标量，这个数值就是 Q 值 $Q(s,a)$。

  假设 $Q(s,a)$ 定义成这个样子，我们要怎么找到一个$a$去最大化这个 Q 值呢？这个方案非常简单。因为 $(a-\mu(s))^{T} \Sigma(s)(a-\mu(s))$ 一定是正的，它前面乘上一个负号，所以第一项就假设我们不看这个负号的话，第一项的值越小，最终的 Q 值就越大。因为我们是把 $V(s)$ 减掉第一项，所以第一项的值越小，最后的 Q 值就越大。怎么让第一项的值最小呢？你直接把$a$代入 $\mu$ 的值，让它变成 0，就会让第一项的值最小。

  $\Sigma$ 一定是正定的。因为这个东西就像是高斯分布（Gaussian distribution），所以 $\mu$ 就是高斯分布的均值，$\Sigma$ 就是高斯分布的方差。但方差是一个正定（positive definite）的矩阵，怎么样让这个 $\Sigma$ 一定是正定的矩阵呢？其实在 $Q^{\pi}$ 里面，它不是直接输出 $\Sigma$，如果直接输出 一个 $\Sigma$， 它不一定是正定的矩阵。它其实是输出 一个矩阵，然后再把那个矩阵跟另外一个矩阵做转置相乘， 然后可以确保 $\Sigma$ 是正定的。这边要强调的点就是说，实际上它不是直接输出一个矩阵。你再去那个论文里面查看一下它的技巧，它可以保证说 $\Sigma$ 是正定的。

  你把$a$代入 $\mu(s)$ 以后，你可以让 Q 的值最大。所以假设要你 arg max Q 函数，如下式所示。
$$
\mu(s)=\arg \max _{a} Q(s, a)
$$

虽然一般而言，若 Q 是一个一般的函数， 你很难算，但是我们这边设计了 Q 这个函数，$a$ 只要设 $\mu(s)$，我们就得到最大值。你在解这个 arg max 的问题的时候就变得非常容易。所以 DQN 也可以用在连续的情况，只是有一些局限，就是函数不能够随便乱设，它必须有一些限制。

##  方案 4：不使用DQN
第 4 招就是不要用 DQN。用 DQN 处理连续动作还是比较麻烦。
基于策略的方法 PPO 和基于价值的方法 DQN，这两者其实是可以结合在一起的，如下图所示，也就是演员-评论员的方法。

![](img/8.3.png)

# Actor-Critic

## Actor-Critic

在 REINFORCE 算法中，每次需要根据一个策略采集一条完整的轨迹，并计算这条轨迹上的回报。这种采样方式的方差比较大，学习效率也比较低。我们可以借鉴时序差分学习的思想，使用动态规划方法来提高采样的效率，即从状态 $s$ 开始的总回报可以通过当前动作的即时奖励 $r(s,a,s')$ 和下一个状态 $s'$ 的值函数来近似估计。

`演员-评论家算法(Actor-Critic Algorithm)`是一种结合`策略梯度`和`时序差分学习`的强化学习方法，其中：

* 演员(Actor)是指策略函数 $\pi_{\theta}(a|s)$，即学习一个策略来得到尽量高的回报。
* 评论家(Critic)是指值函数 $V^{\pi}(s)$，对当前策略的值函数进行估计，即评估演员的好坏。
* 借助于值函数，演员-评论家算法可以进行单步更新参数，不需要等到回合结束才进行更新。

在  Actor-Critic 算法 里面，最知名的方法就是 `A3C(Asynchronous Advantage Actor-Critic)`。

* 如果去掉 Asynchronous，只有 `Advantage Actor-Critic`，就叫做 `A2C`。
* 如果加了 Asynchronous，变成 `Asynchronous Advantage Actor-Critic`，就变成 `A3C`。

### Review: Policy Gradient

![](img/9.1.png)

那我们复习一下 policy gradient，在 policy gradient，我们在更新 policy 的参数 $\theta$ 的时候，我们是用了下面这个式子来算出 gradient。
$$
\nabla \bar{R}_{\theta} \approx \frac{1}{N} \sum_{n=1}^{N} \sum_{t=1}^{T_{n}}\left(\sum_{t^{\prime}=t}^{T_{n}} \gamma^{t^{\prime}-t} r_{t^{\prime}}^{n}-b\right) \nabla \log p_{\theta}\left(a_{t}^{n} \mid s_{t}^{n}\right)
$$
这个式子是在说，我们先让 agent 去跟环境互动一下，那我们可以计算出在某一个状态 s，采取了某一个动作 a 的概率  $p_{\theta}(a_t|s_t)$。接下来，我们去计算在某一个状态 s 采取了某一个动作 a 之后，到游戏结束为止，累积奖励有多大。我们把这些奖励从时间 t 到时间 T 的奖励通通加起来，并且会在前面乘一个折扣因子，可能设 0.9 或 0.99。我们会减掉一个 baseline b，减掉这个值 b 的目的，是希望括号这里面这一项是有正有负的。如果括号里面这一项是正的，我们就要增加在这个状态采取这个动作的机率；如果括号里面是负的，我们就要减少在这个状态采取这个动作的机率。

我们把用 G 来表示累积奖励。但 G 这个值，其实是非常不稳定的。因为互动的过程本身是有随机性的，所以在某一个状态 s 采取某一个动作 a，然后计算累积奖励，每次算出来的结果都是不一样的，所以 G 其实是一个随机变量。给同样的状态 s，给同样的动作 a，G 可能有一个固定的分布。但我们是采取采样的方式，我们在某一个状态 s 采取某一个动作 a，然后玩到底，我们看看得到多少的奖励，我们就把这个东西当作 G。

把 G 想成是一个随机变量的话，我们实际上是对这个 G 做一些采样，然后拿这些采样的结果，去更新我们的参数。但实际上在某一个状态 s 采取某一个动作 a，接下来会发生什么事，它本身是有随机性的。虽然说有个固定的分布，但它本身是有随机性的，而这个随机变量的方差可能会非常大。你在同一个状态采取同一个动作，你最后得到的结果可能会是天差地远的。

假设我们可以采样足够的次数，在每次更新参数之前，我们都可以采样足够的次数，那其实没有什么问题。但问题就是我们每次做 policy gradient，每次更新参数之前都要做一些采样，这个采样的次数其实是不可能太多的，我们只能够做非常少量的采样。如果你正好采样到差的结果，比如说你采样到 G = 100，采样到 G = -10，那显然你的结果会是很差的。

### Review: Q-learning

![](img/9.2.png)

Q: 能不能让整个训练过程变得比较稳定一点，能不能够直接估测 G 这个随机变量的期望值？

A: 我们在状态 s 采取动作 a 的时候，直接用一个网络去估测在状态 s 采取动作 a 的时候，G 的期望值。如果这件事情是可行的，那之后训练的时候，就用期望值来代替采样的值，这样会让训练变得比较稳定。

Q: 怎么拿期望值代替采样的值呢？

A: 这边就需要引入基于价值的(value-based)的方法。基于价值的方法就是 Q-learning。Q-learning 有两种函数，有两种 critics。

* 第一种 critic 是 $V^{\pi}(s)$，它的意思是说，假设 actor 是 $\pi$，拿 $\pi$ 去跟环境做互动，当我们看到状态 s 的时候，接下来累积奖励 的期望值有多少。
* 还有一个 critic 是 $Q^{\pi}(s,a)$。$Q^{\pi}(s,a)$ 把 s 跟 a 当作输入，它的意思是说，在状态 s 采取动作 a，接下来都用 actor $\pi$ 来跟环境进行互动，累积奖励的期望值是多少。

* $V^{\pi}$ 输入 s，输出一个标量。

* $Q^{\pi}$ 输入 s，然后它会给每一个 a 都分配一个 Q value。

* 你可以用  TD 或 MC 来估计。用 TD 比较稳，用 MC 比较精确。

### Actor-Critic

![](img/9.3.png)

随机变量 $G$ 的期望值正好就是 Q ，即
$$
E\left[G_{t}^{n}\right]=Q^{\pi_{\theta}} \left(s_{t}^{n}, a_{t}^{n}\right)
$$

因为这个就是 Q 的定义。Q-function 的定义就是在某一个状态 s，采取某一个动作 a，假设 policy 就是 $\pi$ 的情况下会得到的累积奖励的期望值有多大，而这个东西就是 G 的期望值。累积奖励的期望值就是 G 的期望值。

所以假设用 $E\left[G_{t}^{n}\right]$ 来代表 $\sum_{t^{\prime}=t}^{T_{n}} \gamma^{t^{\prime}-t} r_{t^{\prime}}^{n}$ 这一项的话，把 Q-function 套在这里就结束了，我们就可以把 Actor 跟 Critic 这两个方法结合起来。

有不同的方法来表示 baseline，但一个常见的做法是用价值函数 $V^{\pi_{\theta}}\left(s_{t}^{n}\right)$ 来表示 baseline。价值函数是说，假设 policy 是 $\pi$，在某一个状态 s 一直互动到游戏结束，期望奖励(expected reward)有多大。 $V^{\pi_{\theta}}\left(s_{t}^{n}\right)$ 没有涉及到动作，$Q^{\pi_{\theta}}\left(s_{t}^{n}, a_{t}^{n}\right)$ 涉及到动作。

其实 $V^{\pi_{\theta}}\left(s_{t}^{n}\right)$ 会是 $Q^{\pi_{\theta}}\left(s_{t}^{n}, a_{t}^{n}\right)$ 的期望值，所以 $Q^{\pi_{\theta}}\left(s_{t}^{n}, a_{t}^{n}\right)-V^{\pi_{\theta}}\left(s_{t}^{n}\right)$ 会有正有负，所以 $\sum_{t^{\prime}=t}^{T_{n}} \gamma^{t^{\prime}-t} r_{t^{\prime}}^{n}-b$ 这一项就会是有正有负的。

所以我们就把 policy gradient 里面 $\sum_{t^{\prime}=t}^{T_{n}} \gamma^{t^{\prime}-t} r_{t^{\prime}}^{n}-b$ 这一项换成了 $Q^{\pi_{\theta}}\left(s_{t}^{n}, a_{t}^{n}\right)-V^{\pi_{\theta}}\left(s_{t}^{n}\right)$。

### Advantage Actor-Critic

![](img/9.4.png)

如果你这么实现的话，有一个缺点是：你要估计 2 个 网络：Q-network 和 V-network，你估测不准的风险就变成两倍。所以我们何不只估测一个网络？

事实上在这个 Actor-Critic 方法里面。你可以只估测 V 这个网络，你可以用 V 的值来表示 Q 的值，$Q^{\pi}\left(s_{t}^{n}, a_{t}^{n}\right)$ 可以写成 $ r_{t}^{n}+V^{\pi}\left(s_{t+1}^{n}\right)$ 的期望值，即
$$
Q^{\pi}\left(s_{t}^{n}, a_{t}^{n}\right)=E\left[r_{t}^{n}+V^{\pi}\left(s_{t+1}^{n}\right)\right]
$$

你在状态 s 采取动作 a，会得到奖励 r，然后跳到状态 $s_{t+1}$。但是你会得到什么样的奖励 r，跳到什么样的状态 $s_{t+1}$，它本身是有随机性的。所以要把右边这个式子，取期望值它才会等于 Q-function。但我们现在把期望值这件事情去掉，即
$$
Q^{\pi}\left(s_{t}^{n}, a_{t}^{n}\right)=r_{t}^{n}+V^{\pi}\left(s_{t+1}^{n}\right)
$$

我们就可以把 Q-function 用 r + V 取代掉，然后得到下式：
$$
r_{t}^{n}+V^{\pi}\left(s_{t+1}^{n}\right)-V^{\pi}\left(s_{t}^{n}\right)
$$
把这个期望值去掉的好处就是你不需要估计 Q 了，你只需要估计 V 就够了，你只要估计 一个网络就够了。但这样你就引入了一个随机的东西 r ，它是有随机性的，它是一个随机变量。但是这个随机变量，相较于累积奖励 G 可能还好，因为它是某一个步骤会得到的奖励，而 G 是所有未来会得到的奖励的总和。G 的方差比较大，r 虽然也有一些方差，但它的方差会比 G 要小。所以把原来方差比较大的 G 换成方差比较小的 r 也是合理的。

Q: 为什么可以直接把期望值拿掉？

A: 原始的 A3C paper 试了各种方法，最后做出来就是这个最好。当然你可能说，搞不好估计 Q 和 V，也可以估计 很好，那我告诉你就是做实验的时候，最后结果就是这个最好，所以后来大家都用这个。

![](img/9.5.png)

因为 $r_{t}^{n}+V^{\pi}\left(s_{t+1}^{n}\right)-V^{\pi}\left(s_{t}^{n}\right)$ 叫做 `Advantage function`。所以这整个方法就叫 `Advantage Actor-Critic`。

整个流程是这样子的。我们有一个 $\pi$，有个初始的 actor 去跟环境做互动，先收集资料。在 policy gradient 方法里面收集资料以后，你就要拿去更新 policy。但是在 actor-critic 方法里面，你不是直接拿那些资料去更新 policy。你先拿这些资料去估计价值函数，你可以用 TD 或 MC 来估计价值函数 。接下来，你再基于价值函数，套用下面这个式子去更新 $\pi$。
$$
\nabla \bar{R}_{\theta} \approx \frac{1}{N} \sum_{n=1}^{N} \sum_{t=1}^{T_{n}}\left(r_{t}^{n}+V^{\pi}\left(s_{t+1}^{n}\right)-V^{\pi}\left(s_{t}^{n}\right)\right) \nabla \log p_{\theta}\left(a_{t}^{n} \mid s_{t}^{n}\right)
$$
然后你有了新的 $\pi$ 以后，再去跟环境互动，再收集新的资料，去估计价值函数。然后再用新的价值函数 去更新 policy，去更新 actor。

整个 actor-critic 的算法就是这么运作的。

![](img/9.6.png)

实现 Actor-Critic 的时候，有两个一定会用的 tip。

* 第一个 tip 是说，我们需要估计两个网络：V function 和 policy 的网络（也就是 actor）。
  *  Critic 网络 $V^\pi(s)$ 输入一个状态，输出一个标量。
  *  Actor 网络 $\pi(s)$ 输入一个状态，
     * 如果动作是离散的，输出就是一个动作的分布。
     * 如果动作是连续的，输出就是一个连续的向量。
  * 上图是举的是离散的例子，但连续的情况也是一样的。输入一个状态，然后它决定你现在要采取哪一个动作。**这两个网络，actor 和 critic 的输入都是 s，所以它们前面几个层(layer)，其实是可以共享的。**
    * 尤其是假设你今天是玩 Atari 游戏，输入都是图像。输入的图像都非常复杂，图像很大，通常你前面都会用一些 CNN 来处理，把那些图像抽象成高级(high level)的信息。把像素级别的信息抽象成高级信息这件事情，其实对 actor 跟 critic 来说是可以共用的。所以通常你会让 actor 跟 critic 的共享前面几个层，你会让 actor 跟 critic 的前面几个层共用同一组参数，那这一组参数可能是 CNN 的参数。
    * 先把输入的像素变成比较高级的信息，然后再给 actor 去决定说它要采取什么样的行为，给这个 critic，给价值函数去计算期望奖励。
* **第二个 tip 是我们一样需要探索(exploration)的机制。**在做 Actor-Critic 的时候，有一个常见的探索的方法是你会对你的 $\pi$ 的输出的分布下一个约束。这个约束是希望这个分布的熵(entropy)不要太小，希望这个分布的熵可以大一点，也就是希望不同的动作它的被采用的概率平均一点。这样在测试的时候，它才会多尝试各种不同的动作，才会把这个环境探索的比较好，才会得到比较好的结果。

这个就是 Advantage Actor-Critic。

## A3C
![](img/9.7.png)

强化学习有一个问题就是它很慢，那怎么增加训练的速度呢？举个例子，火影忍者就是有一次鸣人说，他想要在一周之内打败晓，所以要加快修行的速度，他老师就教他一个方法：用影分身进行同样修行。两个一起修行的话，经验值累积的速度就会变成 2 倍，所以鸣人就开了 1000 个影分身来进行修行。这个其实就是 `Asynchronous(异步的) Advantage Actor-Critic`，也就是 A3C 这个方法的精神。

![](img/9.8.png)

**A3C 这个方法就是同时开很多个 worker，那每一个 worker 其实就是一个影分身。那最后这些影分身会把所有的经验，通通集合在一起。**你如果没有很多个 CPU，可能也是不好实现的，你可以实现 A2C 就好。

Q: A3C 是怎么运作的？

A: 

* A3C 一开始有一个 global network。那我们刚才有讲过，其实 policy network 跟 value network 是绑(tie)在一起的，它们的前几个层会被绑一起。我们有一个 global network，它们有包含 policy 的部分和 value 的部分。

* 假设 global network 的参数是 $\theta_1$，你会开很多个 worker。每一个 worker 就用一张 CPU 去跑。比如你就开 8 个 worker，那你至少 8 张 CPU。每一个 worker 工作前都会 global network 的参数复制过来。
* 接下来你就去跟环境做互动，每一个 actor 去跟环境做互动的时候，要收集到比较多样性的数据。举例来说，如果是走迷宫的话，可能每一个 actor 起始的位置都会不一样，这样它们才能够收集到比较多样性的数据。
* 每一个 actor 跟环境做互动，互动完之后，你就会计算出梯度。计算出梯度以后，你要拿梯度去更新你的参数。你就计算一下你的梯度，然后用你的梯度去更新 global network 的参数。就是这个 worker 算出梯度以后，就把梯度传回给中央的控制中心，然后中央的控制中心就会拿这个梯度去更新原来的参数。
* 注意，所有的 actor 都是平行跑的，每一个 actor 就是各做各的，不管彼此。所以每个人都是去要了一个参数以后，做完就把参数传回去。所以当第一个 worker 做完想要把参数传回去的时候，本来它要的参数是 $\theta_1$，等它要把梯度传回去的时候。可能别人已经把原来的参数覆盖掉，变成 $\theta_2$了。但是没有关系，它一样会把这个梯度就覆盖过去就是了。Asynchronous actor-critic 就是这么做的，这个就是 A3C。

 ## Pathwise Derivative Policy Gradient
![](img/9.9.png)

讲完 A3C 之后，我们要讲另外一个方法叫做 `Pathwise Derivative Policy Gradient`。这个方法可以看成是 Q-learning 解连续动作的一种特别的方法，也可以看成是一种特别的 Actor-Critic 的方法。

用棋灵王来比喻的话，阿光是一个 actor，佐为是一个 critic。阿光落某一子以后，

* 如果佐为是一般的 Actor-Critic，他会告诉阿光说这时候不应该下小马步飞，他会告诉你，你现在采取的这一步算出来的 value 到底是好还是不好，但这样就结束了，他只告诉你说好还是不好。因为一般的这个 Actor-Critic 里面那个 critic 就是输入状态或输入状态跟动作的对(pair)，然后给你一个 value 就结束了。所以对 actor 来说，它只知道它做的这个行为到底是好还是不好。
* 但如果是在 pathwise derivative policy gradient 里面，这个 critic 会直接告诉 actor 说采取什么样的动作才是好的。所以今天佐为不只是告诉阿光说，这个时候不要下小马步飞，同时还告诉阿光说这个时候应该要下大马步飞，所以这个就是 Pathwise Derivative Policy Gradient 中的 critic。critic 会直接告诉 actor 做什么样的动作才可以得到比较大的 value。

从 Q-learning 的观点来看，Q-learning 的一个问题是你在用 Q-learning 的时候，考虑 continuous vector 会比较麻烦，比较没有通用的解决方法(general solution)，怎么解这个优化问题呢？

我们用一个 actor 来解这个优化的问题。本来在 Q-learning 里面，如果是一个连续的动作，我们要解这个优化问题。但是现在这个优化问题由 actor 来解，假设 actor 就是一个 solver，这个 solver 的工作就是给定状态 s，然后它就去解，告诉我们说，哪一个动作可以给我们最大的 Q value，这是从另外一个观点来看 pathwise derivative policy gradient 这件事情。

在 GAN 中也有类似的说法。我们学习一个 discriminator 来评估东西好不好，要 discriminator 生成东西的话，非常困难，那怎么办？因为要解一个 arg max 的问题非常的困难，所以用 generator 来生成。

所以今天的概念其实是一样的，Q 就是那个 discriminator。要根据这个 discriminator 决定动作非常困难，怎么办？另外学习一个网络来解这个优化问题，这个东西就是 actor。

所以两个不同的观点是同一件事。从两个不同的观点来看，

* 一个观点是说，我们可以对原来的 Q-learning 加以改进，我们学习一个 actor 来决定动作以解决 arg max 不好解的问题。
* 另外一个观点是，原来的 actor-critic 的问题是 critic 并没有给 actor 足够的信息，它只告诉它好或不好，没有告诉它说什么样叫好，那现在有新的方法可以直接告诉 actor 说，什么样叫做好。

![](img/9.10.png)

那我们讲一下它的算法。假设我们学习了一个 Q-function，Q-function 就是输入 s 跟 a，输出就是 $Q^{\pi}(s,a)$。那接下来，我们要学习一个 actor，这个 actor 的工作就是解这个 arg max 的问题。这个 actor 的工作就是输入一个状态 s，希望可以输出一个动作 a。这个动作 a 被丢到 Q-function 以后，它可以让 $Q^{\pi}(s,a)$ 的值越大越好。

那实际上在训练的时候，你其实就是把 Q 跟 actor 接起来变成一个比较大的网络。Q 是一个网络，输入 s 跟 a，输出一个 value。Actor 在训练的时候，它要做的事情就是输入 s，输出 a。把 a 丢到 Q 里面，希望输出的值越大越好。在训练的时候会把 Q 跟 actor 接起来，当作是一个大的网络。然后你会固定住 Q 的参数，只去调 actor 的参数，就用 gradient ascent 的方法去最大化 Q 的输出。这就是一个 GAN，这就是 conditional GAN。Q 就是 discriminator，但在强化学习就是 critic，actor 在 GAN 里面就是 generator，其实它们就是同一件事情。

![](img/9.11.png)

我们来看一下 pathwise derivative policy gradient 的算法。一开始你会有一个 actor $\pi$，它去跟环境互动，然后，你可能会要它去估计 Q value。估计完 Q value 以后，你就把 Q value 固定，只去学习一个 actor。假设这个 Q 估得是很准的，它知道在某一个状态采取什么样的动作，会真的得到很大的 value。接下来就学习这个 actor，actor 在给定 s 的时候，它采取了 a，可以让最后 Q-function 算出来的 value 越大越好。你用这个 criteria 去更新你的 actor $\pi$。然后有新的 $\pi$ 再去跟环境做互动，再估计 Q，再得到新的 $\pi$ 去最大化 Q 的输出。本来在 Q-learning 里面，你用得上的技巧，在这边也几乎都用得上，比如说 replay buffer、exploration 等等。

![](img/9.12.png)

上图是原来 Q-learning 的算法。你有一个 Q-function Q，你有另外一个目标的 Q-function 叫做 $\hat{Q}$。然后在每一次 训练，在每一个回合的每一个时间点里面，你会看到一个状态 $s_t$，你会采取某一个动作 $a_{t}$。至于采取哪一个动作是由 Q-function 所决定的，因为解一个 arg max 的问题。如果是离散的话没有问题，你就看说哪一个 a 可以让 Q 的 value 最大，就采取哪一个动作。那你需要加一些探索，这样表现才会好。你会得到奖励 $r_t$，跳到新的状态 $s_{t+1}$。你会把 $s_t$, $a_{t}$, $r_t$, $s_{t+1}$ 塞到你的 buffer 里面去。你会从你的 buffer 里面采样一个批量的数据，在这个批量数据里面，可能某一笔是 $s_i, a_i, r_i, s_{i+1}$。接下来你会算一个目标，这个目标叫做 $y$ ，$y=r_{i}+\max _{a} \hat{Q}\left(s_{i+1}, a\right)$。然后怎么学习你的 Q 呢？你希望 $Q(s_i,a_i)$ 跟 y 越接近越好，这是一个回归的问题，最后每 C 个步骤，你要把用 Q 替代 $\hat{Q}$ 。

![](img/9.13.png)

 接下来我们把 Q-learning 改成 Pathwise Derivative Policy Gradient，这边需要做四个改变。

* 第一个改变是，你要把 Q 换成 $\pi$，本来是用 Q 来决定在状态 $s_t$ 产生那一个动作, $a_{t}$ 现在是直接用 $\pi$ 。我们不用再解 arg max 的问题了，我们直接学习了一个 actor。这个 actor 输入 $s_t$ 就会告诉我们应该采取哪一个 $a_{t}$。所以本来输入 $s_t$，采取哪一个 $a_t$，是 Q 决定的。在 Pathwise Derivative Policy Gradient 里面，我们会直接用 $\pi$ 来决定，这是第一个改变。
* 第二个改变是，本来这个地方是要计算在 $s_{i+1}$，根据你的 policy 采取某一个动作 a 会得到多少的 Q value。那你会采取让 $\hat{Q}$ 最大的那个动作 a。那现在因为我们其实不好解这个 arg max 的问题，所以 arg max 问题，其实现在就是由 policy $\pi$ 来解了，所以我们就直接把 $s_{i+1}$ 代到 policy $\pi$ 里面，你就会知道说给定 $s_{i+1}$ ，哪一个动作会给我们最大的 Q value，那你在这边就会采取那一个动作。在 Q-function 里面，有两个 Q network，一个是真正的 Q network，另外一个是目标 Q network。那实际上你在实现这个算法 的时候，你也会有两个 actor，你会有一个真正要学习的 actor $\pi$，你会有一个目标 actor $\hat{\pi}$ 。这个原理就跟为什么要有目标 Q network 一样，我们在算目标 value 的时候，我们并不希望它一直的变动，所以我们会有一个目标的 actor 和一个目标的 Q-function，它们平常的参数就是固定住的，这样可以让你的这个目标的 value 不会一直地变化。所以本来到底是要用哪一个动作 a，你会看说哪一个动作 a 可以让 $\hat{Q}$  最大。但现在因为哪一个动作 a 可以让 $\hat{Q}$ 最大这件事情已经用 policy 取代掉了，所以我们要知道哪一个动作 a 可以让 $\hat{Q}$ 最大，就直接把那个状态带到 $\hat{\pi}$ 里面，看它得到哪一个 a，那个 a 就是会让 $\hat{Q}(s,a)$ 的值最大的那个 a 。其实跟原来的这个 Q-learning 也是没什么不同，只是原来你要解 arg max 的地方，通通都用 policy 取代掉了，那这个是第二个不同。
* 第三个不同就是之前只要学习 Q，现在你多学习一个 $\pi$，那学习 $\pi$ 的时候的方向是什么呢？学习 $\pi$ 的目的，就是为了最大化 Q-function，希望你得到的这个 actor，它可以让你的 Q-function 输出越大越好，这个跟学习 GAN 里面的 generator 的概念。其实是一样的。
* 第四个步骤，就跟原来的 Q-function 一样。你要把目标的 Q network 取代掉，你现在也要把目标 policy 取代掉。

## Connection with GAN
![](img/9.14.png)

其实 GAN 跟 Actor-Critic 的方法是非常类似的。这边就不细讲，你可以去找到一篇 paper 叫做 `Connecting Generative Adversarial Network and Actor-Critic Methods`。

Q: 知道 GAN 跟 Actor-Critic 非常像有什么帮助呢？

A: 一个很大的帮助就是 GAN 跟 Actor-Critic 都是以难训练而闻名的。所以在文献上就会收集各式各样的方法，告诉你说怎么样可以把 GAN 训练起来。怎么样可以把 Actor-Critic 训练起来。但是因为做 GAN 跟 Actor-Critic 的人是两群人，所以这篇 paper 里面就列出说在 GAN 上面有哪些技术是有人做过的，在 Actor-Critic 上面，有哪些技术是有人做过的。也许在 GAN 上面有试过的技术，你可以试着应用在 Actor-Critic 上，在 Actor-Critic 上面做过的技术，你可以试着应用在 GAN 上面，看看是否 work。

## 2 Questions

- 为什么在DQN中采用价值函数近似（Value Function Approximation）的表示方法？

  答：首先DQN为基于深度学习的Q-learning算法，而在Q-learning中，我们使用表格来存储每一个state下action的reward，即我们前面所讲的状态-动作值函数 $Q(s,a)$ 。但是在我们的实际任务中，状态量通常数量巨大并且在连续的任务中，会遇到维度灾难的问题，所以使用真正的Value Function通常是不切实际的，所以使用了价值函数近似（Value Function Approximation）的表示方法。

- critic output通常与哪几个值直接相关？

  答：critic output与state和actor有关。我们在讨论output时通常是对于一个actor下来衡量一个state的好坏，也就是state value本质上来说是依赖于actor。不同的actor在相同的state下也会有不同的output。

- 我们通常怎么衡量state value function  $V^{\pi}(s)$ ?分别的优势和劣势有哪些？

  答：

  - **基于Monte-Carlo（MC）的方法** ：本质上就是让actor与environment做互动。critic根据”统计“的结果，将actor和state对应起来，即当actor如果看到某一state $s_a$ ，将预测接下来的accumulated reward有多大如果它看到 state $s_b$，接下来accumulated reward 会有多大。 但是因为其普适性不好，其需要把所有的state都匹配到，如果我们我们是做一个简单的贪吃蛇游戏等state有限的问题，还可以进行。但是如果我们做的是一个图片型的任务，我们几乎不可能将所有的state（对应每一帧的图像）的都”记录“下来。总之，其不能对于未出现过的input state进行对应的value的输出。
  - **基于MC的Network方法：** 为了解决上面描述的Monte-Carlo（MC）方法的不足，我们将其中的state value function  $V^{\pi}(s)$ 定义为一个Network，其可以对于从未出现过的input state，根据network的泛化和拟合能力，也可以”估测“出一个value output。
  - **基于Temporal-difference（时序差分）的Network方法，即TD based Network：** 与我们再前4章介绍的MC与TD的区别一样，这里两者的区别也相同。在 MC based 的方法中，每次我们都要算 accumulated reward，也就是从某一个 state $s_a$ 一直玩到游戏结束的时候，得到的所有 reward 的总和。所以要应用 MC based 方法时，我们必须至少把这个游戏玩到结束。但有些游戏非常的长，你要玩到游戏结束才能够 update network，花的时间太长了。因此我们会采用 TD based 的方法。TD based 的方法不需要把游戏玩到底，只要在游戏的某一个情况，某一个 state $s_t$ 的时候，采取 action $a_t$ 得到 reward $r_t$ ，跳到 state $s_{t+1}$，就可以应用 TD 的方法。公式与之前介绍的TD方法类似，即：$V^{\pi}\left(s_{t}\right)=V^{\pi}\left(s_{t+1}\right)+r_{t}$。
  - **基于MC和基于TD的区别在于：** MC本身具有很大的随机性，我们可以将其 $G_a$  堪称一个random的变量，所以其最终的variance很大。而对于TD，其具有随机性的变量为 $r$ ,因为计算 $s_t$ 我们采取同一个 action，你得到的 reward 也不一定是一样的，所以对于TD来说，$r$ 是一个 random 变量。但是相对于MC的 $G_a$  的随机程度来说， $r$ 的随机性非常小，这是因为本身 $G_a$ 就是由很多的 $r$ 组合而成的。但另一个角度来说， 在TD中，我们的前提是 $r_t=V^{\pi}\left(s_{t+1}\right)-V^{\pi}\left(s_{t}\right)$ ,但是我们通常无法保证 $V^{\pi}\left(s_{t+1}\right)、V^{\pi}\left(s_{t}\right)$ 计算的误差为零。所以当 $V^{\pi}\left(s_{t+1}\right)、V^{\pi}\left(s_{t}\right)$  计算的不准确的话，那应用上式得到的结果，其实也会是不准的。所以 MC 跟 TD各有优劣。
  - **目前， TD 的方法是比较常见的，MC 的方法其实是比较少用的。**

- 基于我们上面说的network（基于MC）的方法，我们怎么训练这个网络呢？或者我们应该将其看做ML中什么类型的问题呢？

  答：理想状态，我们期望对于一个input state输出其无误差的reward value。也就是说这个 value function 来说，如果 input 是 state $s_a$，正确的 output 应该是$G_a$。如果 input state $s_b$，正确的output 应该是value $G_b$。所以在训练的时候，其就是一个典型的ML中的回归问题（regression problem）。所以我们实际中需要输出的仅仅是一个非精确值，即你希望在 input $s_a$ 的时候，output value 跟 $G_a$ 越近越好，input $s_b$ 的时候，output value 跟 $G_b$ 越近越好。其训练方法，和我们在训练CNN、DNN时的方法类似，就不再一一赘述。

- 基于上面介绍的基于TD的network方法，具体地，我们应该怎么训练模型呢？

  答：核心的函数为 $V^{\pi}\left(s_{t}\right)=V^{\pi}\left(s_{t+1}\right)+r_{t}$。我们将state $s_t$  作为input输入network 里，因为 $s_t$ 丢到 network 里面会得到output $V^{\pi}(s_t)$，同样将 $s_{t+1}$ 作为input输入 network 里面会得到$V^{\pi}(s_{t+1})$。同时核心函数：$V^{\pi}\left(s_{t}\right)=V^{\pi}\left(s_{t+1}\right)+r_{t}$  告诉我们，  $V^{\pi}(s_t)$ 减 $V^{\pi}(s_{t+1})$ 的值应该是 $r_t$。然后希望它们两个相减的 loss 跟 $r_t$ 尽可能地接近。这也就是我们这个network的优化目标或者说loss function。

- state-action value function（Q-function）和 state value function的有什么区别和联系？

  答：

  - state value function 的 input 是一个 state，它是根据 state 去计算出，看到这个state 以后的 expected accumulated reward 是多少。
  - state-action value function 的 input 是一个 state 跟 action 的 pair，即在某一个 state 采取某一个action，假设我们都使用 actor $\pi$ ，得到的 accumulated reward 的期望值有多大。

- Q-function的两种表示方法？

  答：

  - 当input 是 state和action的pair时，output 就是一个 scalar。
  - 当input 仅是一个 state时，output 就是好几个 value。

- 当我们有了Q-function后，我们怎么找到更好的策略 $\pi'$ 呢？或者说这个 $\pi'$ 本质来说是什么？

  答：首先， $\pi'$ 是由 $\pi^{\prime}(s)=\arg \max _{a} Q^{\pi}(s, a)$ 计算而得，其表示假设你已经 learn 出 $\pi$ 的Q-function，今天在某一个 state s，把所有可能的 action a 都一一带入这个 Q-function，看看说那一个 a 可以让 Q-function 的 value 最大，那这一个 action，就是 $\pi'$ 会采取的 action。所以根据上式决定的actoin的步骤一定比原来的 $\pi$ 要好，即$V^{\pi^{\prime}}(s) \geq V^{\pi}(s)$。

- 解决`探索-利用窘境(Exploration-Exploitation dilemma)`问题的Exploration的方法有哪些？他们具体的方法是怎样的？

  答：

  1. **Epsilon Greedy：** 我们有$1-\varepsilon$ 的机率，通常 $\varepsilon$ 很小，完全按照Q-function 来决定action。但是有 $\varepsilon$ 的机率是随机的。通常在实现上 $\varepsilon$ 会随着时间递减。也就是在最开始的时候。因为还不知道那个action 是比较好的，所以你会花比较大的力气在做 exploration。接下来随着training 的次数越来越多。已经比较确定说哪一个Q 是比较好的。你就会减少你的exploration，你会把 $\varepsilon$ 的值变小，主要根据Q-function 来决定你的action，比较少做random，这是**Epsilon Greedy**。
  2. **Boltzmann Exploration：** 这个方法就比较像是 policy gradient。在 policy gradient 里面network 的output 是一个 expected action space 上面的一个的 probability distribution。再根据 probability distribution 去做 sample。所以也可以根据Q value 去定一个 probability distribution，假设某一个 action 的 Q value 越大，代表它越好，我们采取这个 action 的机率就越高。这是**Boltzmann Exploration**。

- 我们使用Experience Replay（经验回放）有什么好处？

  答：

  1. 首先，在强化学习的整个过程中， 最花时间的 step 是在跟环境做互动，使用GPU乃至TPU加速来训练 network 相对来说是比较快的。而用 replay buffer 可以减少跟环境做互动的次数，因为在训练的时候，我们的 experience 不需要通通来自于某一个policy（或者当前时刻的policy）。一些过去的 policy 所得到的 experience 可以放在 buffer 里面被使用很多次，被反复的再利用，这样让你的 sample 到 experience 的利用是高效的。
  2. 另外，在训练网络的时候，其实我们希望一个 batch 里面的 data 越 diverse 越好。如果你的 batch 里面的 data 都是同样性质的，我们的训练出的模型拟合能力不会很乐观。如果 batch 里面都是一样的 data，你 train 的时候，performance 会比较差。我们希望 batch data 越 diverse 越好。那如果 buffer 里面的那些 experience 通通来自于不同的 policy ，那你 sample 到的一个 batch 里面的 data 会是比较 diverse 。这样可以保证我们模型的性能至少不会很差。

- 在Experience Replay中我们是要观察 $\pi$ 的 value，里面混杂了一些不是 $\pi$ 的 experience ，这会有影响吗？

  答：没关系。这并不是因为过去的 $\pi$ 跟现在的 $\pi$ 很像， 就算过去的$\pi$ 没有很像，其实也是没有关系的。主要的原因是我们并不是去sample 一个trajectory，我们只sample 了一个experience，所以跟是不是 off-policy 这件事是没有关系的。就算是off-policy，就算是这些 experience 不是来自于 $\pi$，我们其实还是可以拿这些 experience 来估测 $Q^{\pi}(s,a)$。

- DQN（Deep Q-learning）和Q-learning有什么异同点？

  答：整体来说，从名称就可以看出，两者的目标价值以及价值的update方式基本相同，另外一方面，不同点在于：

  - 首先，DQN 将 Q-learning 与深度学习结合，用深度网络来近似动作价值函数，而 Q-learning 则是采用表格存储。
  - DQN 采用了我们前面所描述的经验回放（Experience Replay）训练方法，从历史数据中随机采样，而 Q-learning 直接采用下一个状态的数据进行学习。
  
  
- 为什么传统的DQN的效果并不好？参考公式 $Q(s_t ,a_t)=r_t+\max_{a}Q(s_{t+1},a)$ 

  答：因为实际上在做的时候，是要让左边这个式子跟右边这个 target 越接近越好。比较容易可以发现target 的值很容易一不小心就被设得太高。因为在算这个 target 的时候，我们实际上在做的事情是看哪一个a 可以得到最大的Q value，就把它加上去，就变成我们的target。

  举例来说，现在有 4 个 actions，本来其实它们得到的值都是差不多的，它们得到的reward 都是差不多的。但是在estimate 的时候，那毕竟是个network。所以estimate 的时候是有误差的。所以假设今天是第一个action它被高估了，假设绿色的东西代表是被高估的量，它被高估了，那这个target 就会选这个action。然后就会选这个高估的Q value来加上$r_t$，来当作你的target。如果第4 个action 被高估了，那就会选第4 个action 来加上$r_t$ 来当作你的target value。所以你总是会选那个Q value 被高估的，你总是会选那个reward 被高估的action 当作这个max 的结果去加上$r_t$ 当作你的target。所以你的target 总是太大。

- 接着上个思考题，我们应该怎么解决target 总是太大的问题呢？

  答： 我们可以使用Double DQN解决这个问题。首先，在 Double DQN 里面，选 action 的 Q-function 跟算 value 的 Q-function不同。在原来的DQN 里面，你穷举所有的 a，把每一个a 都带进去， 看哪一个 a 可以给你的 Q value 最高，那你就把那个 Q value 加上$r_t$。但是在 Double DQN 里面，你**有两个 Q-network**，第一个 Q-network，决定哪一个 action 的 Q value 最大，你用第一个 Q-network 去带入所有的 a，去看看哪一个Q value 最大。然后你决定你的action 以后，你的 Q value 是用 $Q'$ 算出来的，这样子有什么好处呢？为什么这样就可以避免 over estimate 的问题呢？因为今天假设我们有两个 Q-function，假设第一个Q-function 它高估了它现在选出来的action a，那没关系，只要第二个Q-function $Q'$ 没有高估这个action a 的值，那你算出来的，就还是正常的值。假设反过来是 $Q'$ 高估了某一个action 的值，那也没差， 因为反正只要前面这个Q 不要选那个action 出来就没事了。

- 哪来 Q  跟 $Q'$ 呢？哪来两个 network 呢？

  答：在实现上，你有两个 Q-network， 一个是 target 的 Q-network，一个是真正你会 update 的 Q-network。所以在 Double DQN 里面，你的实现方法会是拿你会 update 参数的那个 Q-network 去选action，然后你拿target 的network，那个固定住不动的network 去算value。而 Double DQN 相较于原来的 DQN 的更改是最少的，它几乎没有增加任何的运算量，连新的network 都不用，因为你原来就有两个network 了。你唯一要做的事情只有，本来你在找最大的a 的时候，你在决定这个a 要放哪一个的时候，你是用$Q'$ 来算，你是用target network 来算，现在改成用另外一个会 update 的 Q-network 来算。

- 如何理解Dueling DQN的模型变化带来的好处？

  答：对于我们的 $Q(s,a)$ 其对应的state由于为table的形式，所以是离散的，而实际中的state不是离散的。对于 $Q(s,a)$ 的计算公式， $Q(s,a)=V(s)+A(s,a)$ 。其中的 $V(s)$ 是对于不同的state都有值，对于 $A(s,a)$ 对于不同的state都有不同的action对应的值。所以本质上来说，我们最终的矩阵 $Q(s,a)$ 的结果是将每一个 $V(s)$ 加到矩阵 $A(s,a)$ 中得到的。从模型的角度考虑，我们的network直接改变的 $Q(s,a)$ 而是 更改的 $V、A$ 。但是有时我们update时不一定会将 $V(s)$ 和 $Q(s,a)$ 都更新。我们将其分成两个path后，我们就不需要将所有的state-action pair都sample一遍，我们可以使用更高效的estimate Q value方法将最终的 $Q(s,a)$ 计算出来。

- 使用MC和TD平衡方法的优劣分别有哪些？

  答：

  - 优势：因为我们现在 sample 了比较多的step，之前是只sample 了一个step， 所以某一个step 得到的data 是真实值，接下来都是Q value 估测出来的。现在sample 比较多step，sample N 个step 才估测value，所以估测的部分所造成的影响就会比小。
  - 劣势：因为我们的 reward 比较多，当我们把 N 步的 reward 加起来，对应的 variance 就会比较大。但是我们可以选择通过调整 N 值，去在variance 跟不精确的 Q 之间取得一个平衡。这里介绍的参数 N 就是一个hyper parameter，你要调这个N 到底是多少，你是要多 sample 三步，还是多 sample 五步。

- Q-learning相比于policy gradient based方法为什么训练起来效果更好，更平稳？

  答：在 Q-learning 中，只要能够 estimate 出Q-function，就可以保证找到一个比较好的 policy，同样的只要能够 estimate 出 Q-function，就保证可以 improve 对应的 policy。而因为 estimate Q-function 作为一个回归问题，是比较容易的。在这个回归问题中， 我们可以时刻观察我们的模型训练的效果是不是越来越好，一般情况下我们只需要关注 regression 的 loss 有没有下降，你就知道你的 model learn 的好不好。所以 estimate Q-function 相较于 learn 一个 policy 是比较容易的。你只要 estimate Q-function，就可以保证说现在一定会得到比较好的 policy，同样其也比较容易操作。

- Q-learning在处理continuous action时存在什么样的问题呢？

  答：在日常的问题中，我们的问题都是continuous action的，例如我们的 agent 要做的事情是开自驾车，它要决定说它方向盘要左转几度， 右转几度，这就是 continuous 的；假设我们的 agent 是一个机器人，假设它身上有 50 个关节，它的每一个 action 就对应到它身上的这 50 个关节的角度，而那些角度也是 continuous 的。

  然而在解决Q-learning问题时，很重要的一步是要求能够解对应的优化问题。当我们 estimate 出Q-function $Q(s,a)$ 以后,必须要找到一个 action，它可以让 $Q(s,a)$ 最大。假设 action 是 discrete 的，那 a 的可能性都是有限的。但如果action是continuous的情况下，我们就不能像离散的action一样，穷举所有可能的continuous action了。

  为了解决这个问题，有以下几种solutions：

  - 第一个解决方法：我们可以使用所谓的sample方法，即随机sample出N个可能的action，然后一个一个带到我们的Q-function中，计算对应的N个Q value比较哪一个的值最大。但是这个方法因为是sample所以不会非常的精确。
  - 第二个解决方法：我们将这个continuous action问题，看为一个优化问题，从而自然而然地想到了可以用gradient ascend去最大化我们的目标函数。具体地，我们将action看为我们的变量，使用gradient ascend方法去update action对应的Q-value。但是这个方法通常的时间花销比较大，因为是需要迭代运算的。
  - 第三个解决方法：设计一个特别的network架构，设计一个特别的Q-function，使得解我们 argmax Q-value的问题变得非常容易。也就是这边的 Q-function 不是一个 general 的 Q-function，特别设计一下它的样子，让你要找让这个 Q-function 最大的 a 的时候非常容易。但是这个方法的function不能随意乱设，其必须有一些额外的限制。具体的设计方法，可以我们的chapter8的详细教程。
  - 第四个解决方法：不用Q-learning，毕竟用其处理continuous的action比较麻烦。

- 整个Advantage actor-critic（A2C）算法的工作流程是怎样的？

  答：在传统的方法中，我们有一个policy $\pi$ 以及一个初始的actor与environment去做互动，收集数据以及反馈。通过这些每一步得到的数据与反馈，我们就要进一步更新我们的policy $\pi$ ，通常我们所使用的方式是policy gradient。但是对于actor-critic方法，我们不是直接使用每一步得到的数据和反馈进行policy $\pi$ 的更新，而是使用这些数据进行 estimate value function，这里我们通常使用的算法包括前几个chapters重点介绍的TD和MC等算法以及他们的优化算法。接下来我们再基于value function来更新我们的policy，公式如下：
  $$
  \nabla \bar{R}_{\theta} \approx \frac{1}{N} \sum_{n=1}^{N} \sum_{t=1}^{T_{n}}\left(r_{t}^{n}+V^{\pi}\left(s_{t+1}^{n}\right)-V^{\pi}\left(s_{t}^{n}\right)\right) \nabla \log p_{\theta}\left(a_{t}^{n} \mid s_{t}^{n}\right)
  $$
  其中，上式中的 $r_{t}^{n}+V^{\pi}\left(s_{t+1}^{n}\right)-V^{\pi}\left(s_{t}^{n}\right)$ 我们称为Advantage function，我们通过上式得到新的policy后，再去与environment进行交互，然后再重复我们的estimate value function的操作，再用value function来更新我们的policy。以上的整个方法我们称为Advantage Actor-Critic。

- 在实现 Actor-Critic 的时候，有哪些我们用到的tips?

  答：与我们上一章讲述的东西有关：

  1. **estimate 两个 network：** 一个是estimate V function，另外一个是 policy 的 network，也就是你的 actor。 V-network的input 是一个 state，output 是一个 scalar。然后 actor 这个 network的input 是一个 state，output 是一个 action 的 distribution。这两个 network，actor 和 critic 的 input 都是 s，所以它们前面几个 layer，其实是可以 share 的。尤其是假设你今天是玩 Atari 游戏，input 都是 image。那 input 那个 image 都非常复杂，image 很大，通常前面都会用一些 CNN 来处理，把那些 image 抽象成 high level 的 information，所以对 actor 跟 critic 来说是可以共用的。我们可以让 actor 跟 critic 的前面几个 layer 共用同一组参数。那这一组参数可能是 CNN。先把 input 的 pixel 变成比较 high level 的信息，然后再给 actor 去决定说它要采取什么样的行为，给这个 critic，给 value function 去计算 expected reward。
  2. **exploration 机制：** 其目的是对policy $\pi$ 的 output 的分布进行一个限制，从而使得 distribution 的 entropy 不要太小，即希望不同的 action 被采用的机率平均一点。这样在 testing 的时候，它才会多尝试各种不同的 action，才会把这个环境探索的比较好，才会得到比较好的结果。

- A3C（Asynchronous Advantage Actor-Critic）在训练是回有很多的worker进行异步的工作，最后再讲他们所获得的“结果”再集合到一起。那么其具体的如何运作的呢？

  答：A3C一开始会有一个 global network。它们有包含 policy 的部分和 value 的部分，假设它的参数就是 $\theta_1$。对于每一个 worker 都用一张 CPU 训练（举例子说明），第一个 worker 就把 global network 的参数 copy 过来，每一个 worker 工作前都会global network 的参数 copy 过来。然后这个worker就要去跟environment进行交互，每一个 actor 去跟environment做互动后，就会计算出 gradient并且更新global network的参数。这里要注意的是，所有的 actor 都是平行跑的、之间没有交叉。所以每个worker都是在global network“要”了一个参数以后，做完就把参数传回去。所以当第一个 worker 做完想要把参数传回去的时候，本来它要的参数是 $\theta_1$，等它要把 gradient 传回去的时候。可能别人已经把原来的参数覆盖掉，变成 $\theta_2$了。但是没有关系，它一样会把这个 gradient 就覆盖过去就是了。

- 对比经典的Q-learning算法，我们的Pathwise Derivative Policy Gradient有哪些改进之处？

  答：

  1. 首先，把 $Q(s,a)$ 换成 了 $\pi$，之前是用 $Q(s,a)$ 来决定在 state $s_t$ 产生那一个 action, $a_{t}$ 现在是直接用 $\pi$ 。原先我们需要解 argmax 的问题，现在我们直接训练了一个 actor。这个 actor input $s_t$ 就会告诉我们应该采取哪一个 $a_{t}$。综上，本来 input $s_t$，采取哪一个 $a_t$，是 $Q(s,a)$ 决定的。在 Pathwise Derivative Policy Gradient 里面，我们会直接用 $\pi$ 来决定。
  2. 另外，原本是要计算在 $s_{i+1}$ 时对应的 policy 采取的 action a 会得到多少的 Q value，那你会采取让 $\hat{Q}$ 最大的那个 action a。现在因为我们不需要再解argmax 的问题。所以现在我们就直接把 $s_{i+1}$ 代入到 policy $\pi$ 里面，直接就会得到在 $s_{i+1}$ 下，哪一个 action 会给我们最大的 Q value，那你在这边就会 take 那一个 action。在 Q-function 里面，有两个 Q network，一个是真正的 Q network，另外一个是 target Q network。那实际上你在 implement 这个 algorithm 的时候，你也会有两个 actor，你会有一个真正要 learn 的 actor $\pi$，你会有一个 target actor $\hat{\pi}$ 。但现在因为哪一个 action a 可以让 $\hat{Q}$ 最大这件事情已经被用那个 policy 取代掉了，所以我们要知道哪一个 action a 可以让 $\hat{Q}$ 最大，就直接把那个 state 带到 $\hat{\pi}$ 里面，看它得到哪一个 a，就用那一个 a，其也就是会让 $\hat{Q}(s,a)$ 的值最大的那个 a 。
  3. 还有，之前只要 learn Q，现在你多 learn 一个 $\pi$，其目的在于maximize Q-function，希望你得到的这个 actor，它可以让你的 Q-function output 越大越好，这个跟 learn GAN 里面的 generator 的概念类似。
  4. 最后，与原来的 Q-function 一样。我们要把 target 的 Q-network 取代掉，你现在也要把 target policy 取代掉。
  

## Keywords
- **DQN(Deep Q-Network)：**  基于深度学习的Q-learninyang算法，其结合了 Value Function Approximation（价值函数近似）与神经网络技术，并采用了目标网络（Target Network）和经验回放（Experience Replay）等方法进行网络的训练。
- **State-value Function：** 本质是一种critic。其输入为actor某一时刻的state，对应的输出为一个标量，即当actor在对应的state时，预期的到过程结束时间段中获得的value的数值。
- **State-value Function Bellman Equation：** 基于state-value function的Bellman Equation，它表示在状态 $s_t$ 下带来的累积奖励 $G_t$ 的期望。
- **Q-function:** 其也被称为state-action value function。其input 是一个 state 跟 action 的 pair，即在某一个 state 采取某一个action，假设我们都使用 actor $\pi$ ，得到的 accumulated reward 的期望值有多大。
- **Target Network：** 为了解决在基于TD的Network的问题时，优化目标 $\mathrm{Q}^{\pi}\left(s_{t}, a_{t}\right) 
  =r_{t}+\mathrm{Q}^{\pi}\left(s_{t+1}, \pi\left(s_{t+1}\right)\right)$ 左右两侧会同时变化使得训练过程不稳定，从而增大regression的难度。target network选择将上式的右部分即 $r_{t}+\mathrm{Q}^{\pi}\left(s_{t+1}, \pi\left(s_{t+1}\right)\right)$ 固定，通过改变上式左部分的network的参数，进行regression，这也是一个DQN中比较重要的tip。
- **Exploration：**  在我们使用Q-function的时候，我们的policy完全取决于Q-function，有可能导致出现对应的action是固定的某几个数值的情况，而不像policy gradient中的output为随机的，我们再从随机的distribution中sample选择action。这样会导致我们继续训练的input的值一样，从而“加重”output的固定性，导致整个模型的表达能力的急剧下降，这也就是`探索-利用窘境难题(Exploration-Exploitation dilemma)`。所以我们使用`Epsilon Greedy`和 `Boltzmann Exploration`等Exploration方法进行优化。
- **Experience Replay（经验回放）：**  其会构建一个Replay Buffer（Replay Memory），用来保存许多data，每一个data的形式如下：在某一个 state $s_t$，采取某一个action $a_t$，得到了 reward $r_t$，然后跳到 state $s_{t+1}$。我们使用 $\pi$ 去跟环境互动很多次，把收集到的数据都放到这个 replay buffer 中。当我们的buffer”装满“后，就会自动删去最早进入buffer的data。在训练时，对于每一轮迭代都有相对应的batch（与我们训练普通的Network一样通过sample得到），然后用这个batch中的data去update我们的Q-function。综上，Q-function再sample和训练的时候，会用到过去的经验数据，所以这里称这个方法为Experience Replay，其也是DQN中比较重要的tip。
- **Double DQN：** 在Double DQN中存在有两个 Q-network，首先，第一个 Q-network，决定的是哪一个 action 的 Q value 最大，从而决定了你的action。另一方面， Q value 是用 $Q'$ 算出来的，这样就可以避免 over estimate 的问题。具体来说，假设我们有两个 Q-function，假设第一个Q-function 它高估了它现在选出来的action a，那没关系，只要第二个Q-function $Q'$ 没有高估这个action a 的值，那你算出来的，就还是正常的值。
- **Dueling DQN：** 将原来的DQN的计算过程分为**两个path**。对于第一个path，会计算一个于input state有关的一个标量 $V(s)$；对于第二个path，会计算出一个vector $A(s,a)$ ，其对应每一个action。最后的网络是将两个path的结果相加，得到我们最终需要的Q value。用一个公式表示也就是 $Q(s,a)=V(s)+A(s,a)$ 。 
- **Prioritized Experience Replay （优先经验回放）：** 这个方法是为了解决我们在chapter6中提出的**Experience Replay（经验回放）**方法不足进一步优化提出的。我们在使用Experience Replay时是uniformly取出的experience buffer中的sample data，这里并没有考虑数据间的权重大小。例如，我们应该将那些train的效果不好的data对应的权重加大，即其应该有更大的概率被sample到。综上， prioritized experience replay 不仅改变了 sample data 的 distribution，还改变了 training process。
- **Noisy Net：** 其在每一个episode 开始的时候，即要和环境互动的时候，将原来的Q-function 的每一个参数上面加上一个Gaussian noise。那你就把原来的Q-function 变成$\tilde{Q}$ ，即**Noisy Q-function**。同样的我们把每一个network的权重等参数都加上一个Gaussian noise，就得到一个新的network $\tilde{Q}$。我们会使用这个新的network从与环境互动开始到互动结束。
- **Distributional Q-function：** 对于DQN进行model distribution。将最终的网络的output的每一类别的action再进行distribution。
- **Rainbow：** 也就是将我们这两节内容所有的七个tips综合起来的方法，7个方法分别包括：DQN、DDQN、Prioritized DDQN、Dueling DDQN、A3C、Distributional DQN、Noisy DQN，进而考察每一个方法的贡献度或者是否对于与环境的交互式正反馈的。
- **A2C：** Advantage Actor-Critic的缩写，一种Actor-Critic方法。
- **A3C：** Asynchronous（异步的）Advantage Actor-Critic的缩写，一种改进的Actor-Critic方法，通过异步的操作，进行RL模型训练的加速。
-  **Pathwise Derivative Policy Gradient：** 其为使用 Q-learning 解 continuous action 的方法，也是一种 Actor-Critic 方法。其会对于actor提供value最大的action，而不仅仅是提供某一个action的好坏程度。